In [1]:
#WaterClockClass2.py

import copy
import scipy.constants as spc
import pygame
from pygame.key import *
from pygame.locals import *
from pygame.color import *
import pymunk
import pymunk.pygame_util
import math  # for isclose function



class Waterfall(object):
    
    """
    This class implements a simple scene in which there is a static platform (made up of a couple of lines)
    that don't move. Balls appear occasionally and drop onto the platform. They bounce around. 
    pattern is a list of tuples, (index, yi).  This pattern is a relation, not necessarily a function.
    """
    def __init__(self, PhysVar,DimensionVar,pattern, PatternIsDelay = False):


        
        #Game mode
        self.GameMode = -1  # 0 = print function,  1 = draw function
        #Drawing 
        self.IsMouseClicked= False # is mouse 1 clicked?
        self.mouse_pos = []
        #SIMULATION 
        self.kappa = DimVar[0]#scaling variable
        
        
        # Space
        self._space = pymunk.Space()
        #for self.dx, see below Pic and Fountain Dimensions
        self._dt = PhysVar[0]       # Time step
        self._space.gravity = (PhysVar[1][0]*self.kappa, PhysVar[1][1]*self.kappa)
        
        self.ndt = .01  ### #fastest time a nozzle can activate. not equal so simulation's self._dt above
        self.v0 = kappa*PhysVar[2]

        self.tTol = PhysVar[3]       
        self._physics_steps_per_frame = PhysVar[4]   # Number of physics steps per screen frame
        self.DropletMass= PhysVar[5]
        #self.iterations= 15 default is 10.
        #Picture and Fountain Dimensions
        
        self.fWidth = kappa*DimVar[1]
        self.fHeight = kappa*DimVar[2]
        self.xfiller = kappa*DimVar[3]
        self.yfiller = kappa*DimVar[4]
        self.Width = self.fWidth+self.xfiller
        self.Height = self.fHeight+self.yfiller

        self.N = DimVar[5]
        self.nThickOverRad = DimVar[6]
        self.nSpacingOverNozz=DimVar[7]
        self.nLength=kappa*DimVar[8]


        self.nRadius=self.fWidth/2/(self.N+(self.N-1)*self.nSpacingOverNozz)/(1+self.nThickOverRad)
        self.nThickness = self.nRadius*self.nThickOverRad;
        self.nSpacing= 2*(self.nRadius+self.nThickness)*self.nSpacingOverNozz
        
        self.dx = 2*(self.nThickness+self.nRadius)+self.nSpacing  
        
        self.DiscreteY =[] #used for converting function to delay
        yy = self.Height
        ii = 0
        while yy>yfiller:
            self.DiscreteY.append(yy)
            ii+=1
            yy= self.Height+self.v0*self.ndt*ii + self._space.gravity[1]/2*(self.ndt*ii)**2


        #Function Variables
        if not PatternIsDelay :
            self.tdelay = self.ConvertToDelay(pattern) #Delay Function            
        else:            
            self.tdelay = pattern

        self.tempDelay = copy.deepcopy(self.tdelay) # just a copy of the delay function, used in _update_balls
        self.tdelayOriginalInput= copy.deepcopy(self.tdelay)  #another copy to remember original picture, used when switching from a drawn picture to the original view mode.
 
        # pygame
        pygame.init()
        self._screen = pygame.display.set_mode((int(self.Width), int(self.Height))) # fWidth +xfiller X fHeight+yfiller
        self._clock = pygame.time.Clock()
 
        self._draw_options = pymunk.pygame_util.DrawOptions(self._screen)
        self._draw_options.shape_outline_color = self._draw_options.shape_dynamic_color#####!!!!1
 
        # Static barrier walls (lines) that the balls bounce off of
        self._add_static_scenery()
 
        # Balls that exist in the world
        self._balls = []
        # Execution control and time until the next ball spawns
        self._running = True
        self.PicItr = 0  # counts the number of updates to the sim.  resets after all droplets fall.
        self.deletedCounter = 0;  #counts number of droplets deleted.  used to reset droplet sim.



    def run(self):
        """
        The main loop of the game.
        :return: None
        """    
        Tsim = 0; 
        
        

        # Main loop
        self._clear_screen()
        self._draw_objects()
        pygame.display.flip()
        
        
        
        while self._running:
                if self.GameMode == -1:
                    pygame.display.set_caption("Please Select Mode:  0= Set function, 1 = Draw function.")
                    pygame.time.wait(500)
                    self._process_events()
                    
                elif self.GameMode ==1:
                    pygame.display.set_caption("Press and hold mouse to draw.  Press Enter when finished.")
                    self._process_events()
                    

                elif self.GameMode == 0:                                        
                    # Progress time forward
                    for x in range(self._physics_steps_per_frame):
                        self._space.step(self._dt/self._physics_steps_per_frame)
                
                    Tsim+=self._dt
                    self._process_events()
                    self.PicItr+=1;
                    self._update_balls()
                    self._clear_screen()
                    self._draw_objects()
                    pygame.display.flip()
                    # Delay fixed time between frames
                    self._clock.tick(50)
                    TsimString = '%.2f'%(Tsim)
                    pygame.display.set_caption(
                        "Press r to go to Mode Select.  Simulation Time: "+TsimString+"%s| fps: "% (" "*(8-len(TsimString))) +
                        "%.2f"%(self._clock.get_fps()) ) 
                
                else:
                    self.GameMode = -1



                    
                    

    def _add_static_scenery(self):
        """
        Create the static bodies.
        :return: None
        
        if fountain is centered and has 20 nozzles, max width = 4(meters)
        40 tube widths, 20 tube diameters, and 19 spaces in between nozzles.
        """
        
        static_body = self._space.static_body
                
        static_lines = []
        nPos1 = (self.nRadius+self.nThickness,self.fHeight-self.nLength/2.)  #horizontal position of first nozzle
        vertices = [(0+self.xfiller/2.,self.Height),(0+self.xfiller/2.,self.Height-self.nLength),(self.nThickness+self.xfiller/2,self.Height),(self.nThickness+self.xfiller/2,self.Height-self.nLength)]
        for i in range(N):
            static_lines.append( pymunk.Poly(static_body, vertices))
            
            
            addX1 =2*self.nRadius+self.nThickness +i*(2*(self.nRadius+self.nThickness)+self.nSpacing)
            vertices = [(addX1+self.xfiller/2.,self.Height),
                        (addX1+self.xfiller/2.,self.Height-self.nLength),
                        (self.nThickness+self.xfiller/2.+ addX1,self.Height),
                        (self.nThickness+self.xfiller/2.+addX1,self.Height-self.nLength)]
                                           
            static_lines.append( pymunk.Poly(static_body, vertices))
            addX2= (i+1)*(2*(self.nRadius+self.nThickness)+self.nSpacing )
            vertices = [(addX2+self.xfiller/2.,self.Height),
                        (addX2+self.xfiller/2.,self.Height-self.nLength),
                        (self.nThickness+self.xfiller/2.+ addX2,self.Height),
                        (self.nThickness+self.xfiller/2.+addX2,self.Height-self.nLength)]                                                                
        self._space.add(static_lines)

    def _process_events(self):
        """
        Handle game and events like keyboard input. Call once per frame only.
        :return: None
        """
        #MotionCount = 0;
        count = 0;

        if self.GameMode == 0:
            for event in pygame.event.get():
                if event.type == QUIT:
                    self._running = False
                elif event.type == KEYDOWN and event.key == K_ESCAPE:
                    self._running = False
                elif event.type == KEYDOWN and event.key == K_p:
                    pygame.image.save(self._screen, "bouncing_balls.png"  )  
                    
                elif event.type==KEYDOWN and event.key == K_r:
                    self.tdelay =copy.deepcopy( self.tdelayOriginalInput )  
                    self.tempDelay =copy.deepcopy( self.tdelayOriginalInput )
                    self.GameMode =-1
                    self.PicItr = 0;
                    self.deletedCounter=0
                    
                    
                    for i in range(len(self._balls)):   
                        ball = self._balls[i]
                        self._space.remove(ball, ball.body)
                    self._balls = []
                    self._clear_screen()
                    
                    
                
                
        elif self.GameMode ==1: #Draw function mode
            for event in pygame.event.get():
                count+=1
                           
                        
                if event.type ==pygame.MOUSEMOTION and self.IsMouseClicked :
                    pointTuple = pygame.mouse.get_pos()                         
                    point = [pointTuple[0], pointTuple[1]]
                    point[1]= self.Height-point[1] #(converts from pygame coordinates from top left screen corner)

                    if (point[0]>self.xfiller/2 and point[0]<self.Width-self.xfiller/2 and point[1]>yfiller and point[1]<self.Height-self.nLength): #make sure pointer is in fountain screen
                        # get the point aligned with a nozzle.
                        point[0]=int(round((point[0]*1.-self.xfiller/2.-self.nThickness-self.nRadius)/self.dx))                                    
                        
                        self.mouse_pos.append(point)
                        
                        #temporary drawing
                        pygame.draw.circle(self._screen,(0,0,128) ,pointTuple,int(self.nRadius)) #tuple =color.   blue= (0,0,128)
                        
                        pygame.display.flip()
                        
                        
                elif event.type==pygame.MOUSEBUTTONDOWN:
                    self.IsMouseClicked = True
                        
                elif event.type==pygame.MOUSEBUTTONUP:
                    self.IsMouseClicked = False
                                    

                elif event.type == pygame.KEYDOWN : 
                    if event.key==pygame.K_RETURN:   #end draw mode
                        if len(self.mouse_pos)!=0:                            
                            self.tdelay = self.ConvertToDelay(self.mouse_pos, isFromDrawMode = True)
                            self.tempDelay = copy.deepcopy(self.tdelay)
                            self.GameMode = 0;

                            self.mouse_pos.clear() #done so future drawings have a blank canvas.
                            self.PicItr = 0
                            self.deletedcounter= 0
                            self._balls.clear()   
                        
                            
                    elif event.key == K_ESCAPE:  #quit
                        self._running = False
                        

        else:
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key==pygame.K_0:
                        self.GameMode = 0
                    elif event.key == pygame.K_1:
                        self.GameMode=1
                    elif event.key == K_ESCAPE:
                        self._running = False


    def _update_balls(self):
        """
        Create/remove balls as necessary. Call once per frame only.
        :return: None
        """
 
        if (len(self.tempDelay)==0) and self.deletedCounter == len(self.tdelay): # reset variables to repeat pattern
            self.tempDelay = copy.deepcopy(self.tdelay)
            self.PicItr = 0;
            self.deletedCounter=0
        
        i=0;
        
        while i <len(self.tempDelay):        
            if self.tempDelay[i][1] - self.PicItr*self._dt < tTol :   #removed stepsperframe
                self._create_ball(self.tempDelay[i][0])
                del self.tempDelay[i]
            else:
                i+=1
                break;####!!!  #tempDelay is already sorted from smallest delay to largest delay
                
            
            
        # Remove balls that fall below waterline (set at y=yfiller)
        balls_to_remove = [ball for ball in self._balls if ball.body.position.y < self.yfiller-self.nRadius]
        for ball in balls_to_remove:
            self._space.remove(ball, ball.body)
            self._balls.remove(ball)
            self.deletedCounter+=1
        
        

    def _create_ball(self, index ):
        """
        Create a ball.
        :return:
        """
        mass = self.DropletMass
        radius = self.nRadius   #min(self.nRadius, (self.DiscreteY[0]-self.DiscreteY[1])/2 )  #so the droplets are not squished together
        inertia = pymunk.moment_for_circle(mass, 0, radius, (0, 0))
        body = pymunk.Body(mass, inertia)
        
        body.position = self.xfiller/2+ self.nThickness+ self.nRadius+ index*(self.nSpacing+2*(self.nThickness+self.nRadius)), self.Height-self.nLength
        body.angle = spc.pi/2
        
        shape = pymunk.Circle(body, radius, (0, 0))
        self._space.add(body, shape)
        self._balls.append(shape)

    def _clear_screen(self):
        """
        Clears the screen.
        :return: None
        """
        self._screen.fill(THECOLORS["white"])

    def _draw_objects(self):
        """
        Draw the objects.
        :return: None
        """
        self._space.debug_draw(self._draw_options)
        
        
    def ConvertToDelay(self,pattern, isFromDrawMode = False):#convert a relation into time delays for each drop.  
        #returns a list of tuples, (index, tau delay)            
                                      
        
        #discretize y
        for i in range(len(pattern)):
            yDiff = abs(self.DiscreteY[0]-pattern[i][1])
            yIndex = 0
            for j in range(len(self.DiscreteY)):
                if yDiff>abs( self.DiscreteY[j]-pattern[i][1]):
                    yDiff= abs(self.DiscreteY[j]-pattern[i][1])
                    yIndex = j
            pattern[i][1]= self.DiscreteY[yIndex]
            
        
        
        if (isFromDrawMode):# take away duplicates
            pattern2 = [];
            for i in range(len(pattern)):
                isInList=False
                j=0
                while j<len(pattern2) and not isInList:
                    if pattern2[j][0]==pattern[i][0] and math.isclose(pattern2[j][1], pattern[i][1],abs_tol =max(1E-6,2*nRadius)):
                        isInList= True  
                    j+=1
                if not isInList:
                    pattern2.append(pattern[i])
                
        
        pattern = sorted(pattern, key=lambda a: a[1])


        #find the lowest value and calculate T
        pmin = pattern[0][1]
        v0 = self.v0
        g= self._space.gravity[1]
        #time when the water droplets match the pattern perfectly
        T= (-v0-pow(v0**2-2*g*(self.Height-pmin),.5))/g
        
        taui=[] #desired delay times (a relation),  each element = [i, ith tau]
        for ii in range(len(pattern)):
            taui.append((pattern[ii][0] , T - (-v0-pow(v0**2-2*g*(self.Height-pattern[ii][1]),.5))/g ) ) 

        taui = sorted(taui, key=lambda a: a[1])        
                
        return taui


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/RxTenshou/anaconda3/envs/pygame1/lib/python3.6/site-packages/pymunk/libchipmunk.dylib]


In [2]:
#PARAMETERS
kappa = 100.;  # multiplier for the visuals

dt = 1.0 / 150.0 
g= (0,-spc.g)
v0 = -.5  #m/s
tTol =1E-15 #time tolerance for float subtraction
StepsPerFrame = 1 ; # Steps of sim taken every frame
mass = 1;#kg

#fountain (space printer) dimensions
fWidth = 6.5;
fHeight= 6.5; #4*kappa
#border areas
xfiller = 1  # this is coded in as centered, so it's split in half from side to side
yfiller = 0  #2*kappa   #this is coded in as from the bottom.
#Screen Dimensions
Width = fWidth+xfiller
Height = fHeight+yfiller

#nozzle dimensions
N = 50; #number of nozzles
nThickOverRad = 1./10;# ratio between tube thickness and tube radius
nSpacingOverNozz = 2;#spacing over full width of nozzle

nRadius=fWidth/2/(N+(N-1)*nSpacingOverNozz)/(1+nThickOverRad)
nThickness = nRadius*nThickOverRad;
nSpacing= 2*(nRadius+nThickness)*nSpacingOverNozz
nLength = .1;


dx = 2*(nThickness+nRadius)+nSpacing




#Some Arguments for the Function
PhysVar = [dt,g,v0, tTol, StepsPerFrame, mass] # dynamic + time variables
DimVar =[kappa, fWidth,fHeight, xfiller,yfiller , N, nThickOverRad, nSpacingOverNozz,nLength]
#3rd is the function below.







In [3]:

import numpy as np
import matplotlib.pyplot as plt  





##FUNCTIONS ##
#we make x be the last variable in these functions.
#these functions must also only take a list as its argument.

def BottomCircle(a): # CenterPos is a tuple of position
    CenterPos=a[0]
    R=a[1]
    x=a[2]
    return CenterPos[1]-(R**2-(x-CenterPos[0])**2)**.5

def TopCircle(a):
    return -BottomCircle(a)+2*a[0][1]

def Line(a):
    m=a[0]
    b=a[1]
    x=a[2]
    return m*x+b
    
def CenteredAbs(a):
    m=a[0]
    x=a[1]
    return abs(m*(x-fWidth/2))


def CosA(a):
    A=a[0]
    omega = a[1]
    x=a[2]
    return A*np.cos(2*np.pi*omega*(x/fWidth-.5))+ fHeight/2+yfiller
    
    
def asin(a):    
    center=a[0]
    B=a[1]
    omega=a[2]
    x=a[3]
    return center[1]+B*np.arcsin(omega*(x-center[0]))


def negasin(a):return -asin(a)



    
"""discretizes the function and returns a list of lists the elements
being the nozzle number and its y value. formatted for 
the above class. """   
def FuncExpConverter(func,func_args, dx, domain): 
    #domain is tuple of endpoints of func domain.
    f= []
    for i in range(N):
        x= i*dx   ##+xfiller/2.+nThickness + nRadius  #!!!
        if x>= domain[0] and x<=domain[1]:
            func_args[-1]=x       #we make x be the last variable in func_args
            f.append([i,func(func_args)])
    return f 


    

In [4]:

cRadius = fHeight/2.


#_______________________________________________________

#CURVY
curvy = []
curveAmplitude =  fHeight/2
# for i in range(3):
#     curvy+= FuncExpConverter(asin,[(W.fWidth/2,W.fHeight/2+i*curveAmplitude), curveAmplitude/np.pi,2./W.fWidth, 0],W.dx,(0,W.fWidth))

curvy+= FuncExpConverter(asin,[(fWidth/2,fHeight/2), curveAmplitude/np.pi,2./fWidth, 0],dx,(0,fWidth))
curvy+= FuncExpConverter(negasin,[(fWidth/2,-fHeight/2-curveAmplitude), curveAmplitude/np.pi,2./fWidth, 0],dx,(0,fWidth))

# plt.plot(*zip(*curvy), 'r+')
# plt.plot(np.arange(50), curveAmplitude*np.ones(50))
# print(fHeight)

#_______________________________________________________


sign1raw= [(15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (15, 0.0), (16, 0.0), (16, 0.0), (16, 0.0), (16, 0.0), (16, 0.0), (17, 0.0), (17, 0.0), (17, 0.0), (17, 0.0), (17, 0.0), (18, 0.0), (18, 0.0), (18, 0.0), (18, 0.0), (18, 0.0), (19, 0.0), (19, 0.0), (19, 0.0), (19, 0.0), (19, 0.0), (19, 0.0), (20, 0.0), (20, 0.0), (20, 0.0), (20, 0.0), (26, 0.0), (26, 0.0), (26, 0.0), (26, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (29, 0.0), (29, 0.0), (29, 0.0), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (15, 0.010000000000000009), (20, 0.010000000000000009), (20, 0.010000000000000009), (21, 0.010000000000000009), (21, 0.010000000000000009), (21, 0.010000000000000009), (21, 0.010000000000000009), (21, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (29, 0.010000000000000009), (30, 0.010000000000000009), (30, 0.010000000000000009), (30, 0.010000000000000009), (30, 0.010000000000000009), (30, 0.010000000000000009), (16, 0.020000000000000018), (16, 0.020000000000000018), (16, 0.020000000000000018), (15, 0.020000000000000018), (15, 0.020000000000000018), (15, 0.020000000000000018), (15, 0.020000000000000018), (15, 0.020000000000000018), (15, 0.020000000000000018), (22, 0.020000000000000018), (22, 0.020000000000000018), (22, 0.020000000000000018), (22, 0.020000000000000018), (22, 0.020000000000000018), (27, 0.020000000000000018), (27, 0.020000000000000018), (27, 0.020000000000000018), (26, 0.020000000000000018), (26, 0.020000000000000018), (30, 0.020000000000000018), (31, 0.020000000000000018), (31, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (36, 0.020000000000000018), (37, 0.020000000000000018), (37, 0.020000000000000018), (37, 0.020000000000000018), (37, 0.020000000000000018), (37, 0.020000000000000018), (37, 0.020000000000000018), (17, 0.029999999999999805), (16, 0.029999999999999805), (16, 0.029999999999999805), (16, 0.029999999999999805), (22, 0.029999999999999805), (23, 0.029999999999999805), (23, 0.029999999999999805), (23, 0.029999999999999805), (23, 0.029999999999999805), (27, 0.029999999999999805), (27, 0.029999999999999805), (27, 0.029999999999999805), (31, 0.029999999999999805), (31, 0.029999999999999805), (36, 0.029999999999999805), (36, 0.029999999999999805), (36, 0.029999999999999805), (36, 0.029999999999999805), (36, 0.029999999999999805), (36, 0.029999999999999805), (37, 0.029999999999999805), (38, 0.029999999999999805), (38, 0.029999999999999805), (38, 0.029999999999999805), (38, 0.029999999999999805), (38, 0.029999999999999805), (39, 0.029999999999999805), (39, 0.029999999999999805), (39, 0.029999999999999805), (18, 0.040000000000000036), (17, 0.040000000000000036), (17, 0.040000000000000036), (17, 0.040000000000000036), (23, 0.040000000000000036), (23, 0.040000000000000036), (24, 0.040000000000000036), (24, 0.040000000000000036), (27, 0.040000000000000036), (27, 0.040000000000000036), (27, 0.040000000000000036), (31, 0.040000000000000036), (32, 0.040000000000000036), (36, 0.040000000000000036), (36, 0.040000000000000036), (36, 0.040000000000000036), (39, 0.040000000000000036), (39, 0.040000000000000036), (39, 0.040000000000000036), (40, 0.040000000000000036), (18, 0.050000000000000044), (18, 0.050000000000000044), (18, 0.050000000000000044), (24, 0.050000000000000044), (24, 0.050000000000000044), (24, 0.050000000000000044), (27, 0.050000000000000044), (27, 0.050000000000000044), (32, 0.050000000000000044), (36, 0.050000000000000044), (36, 0.050000000000000044), (40, 0.050000000000000044), (40, 0.050000000000000044), (40, 0.050000000000000044), (19, 0.06000000000000005), (19, 0.06000000000000005), (24, 0.06000000000000005), (25, 0.06000000000000005), (28, 0.06000000000000005), (28, 0.06000000000000005), (27, 0.06000000000000005), (27, 0.06000000000000005), (32, 0.06000000000000005), (32, 0.06000000000000005), (32, 0.06000000000000005), (36, 0.06000000000000005), (36, 0.06000000000000005), (40, 0.06000000000000005), (40, 0.06000000000000005), (41, 0.06000000000000005), (41, 0.06000000000000005), (20, 0.07000000000000006), (20, 0.07000000000000006), (19, 0.07000000000000006), (25, 0.07000000000000006), (25, 0.07000000000000006), (25, 0.07000000000000006), (28, 0.07000000000000006), (28, 0.07000000000000006), (28, 0.07000000000000006), (32, 0.07000000000000006), (33, 0.07000000000000006), (33, 0.07000000000000006), (33, 0.07000000000000006), (36, 0.07000000000000006), (36, 0.07000000000000006), (36, 0.07000000000000006), (41, 0.07000000000000006), (41, 0.07000000000000006), (41, 0.07000000000000006), (41, 0.07000000000000006), (21, 0.08000000000000007), (21, 0.08000000000000007), (20, 0.08000000000000007), (26, 0.08000000000000007), (26, 0.08000000000000007), (26, 0.08000000000000007), (28, 0.08000000000000007), (28, 0.08000000000000007), (28, 0.08000000000000007), (28, 0.08000000000000007), (33, 0.08000000000000007), (33, 0.08000000000000007), (33, 0.08000000000000007), (36, 0.08000000000000007), (36, 0.08000000000000007), (41, 0.08000000000000007), (41, 0.08000000000000007), (41, 0.08000000000000007), (41, 0.08000000000000007), (21, 0.09000000000000008), (21, 0.09000000000000008), (26, 0.09000000000000008), (26, 0.09000000000000008), (26, 0.09000000000000008), (26, 0.09000000000000008), (28, 0.09000000000000008), (28, 0.09000000000000008), (28, 0.09000000000000008), (33, 0.09000000000000008), (33, 0.09000000000000008), (33, 0.09000000000000008), (36, 0.09000000000000008), (36, 0.09000000000000008), (36, 0.09000000000000008), (36, 0.09000000000000008), (41, 0.09000000000000008), (41, 0.09000000000000008), (41, 0.09000000000000008), (41, 0.09000000000000008), (22, 0.10000000000000009), (22, 0.10000000000000009), (22, 0.10000000000000009), (27, 0.10000000000000009), (27, 0.10000000000000009), (27, 0.10000000000000009), (27, 0.10000000000000009), (29, 0.10000000000000009), (29, 0.10000000000000009), (28, 0.10000000000000009), (28, 0.10000000000000009), (33, 0.10000000000000009), (33, 0.10000000000000009), (36, 0.10000000000000009), (36, 0.10000000000000009), (36, 0.10000000000000009), (41, 0.10000000000000009), (41, 0.10000000000000009), (41, 0.10000000000000009), (15, 0.11000000000000021), (15, 0.11000000000000021), (15, 0.11000000000000021), (16, 0.11000000000000021), (16, 0.11000000000000021), (16, 0.11000000000000021), (16, 0.11000000000000021), (16, 0.11000000000000021), (23, 0.11000000000000021), (27, 0.11000000000000021), (27, 0.11000000000000021), (27, 0.11000000000000021), (29, 0.11000000000000021), (29, 0.11000000000000021), (33, 0.11000000000000021), (33, 0.11000000000000021), (33, 0.11000000000000021), (36, 0.11000000000000021), (36, 0.11000000000000021), (36, 0.11000000000000021), (41, 0.11000000000000021), (41, 0.11000000000000021), (41, 0.11000000000000021), (12, 0.12000000000000022), (13, 0.12000000000000022), (13, 0.12000000000000022), (13, 0.12000000000000022), (13, 0.12000000000000022), (13, 0.12000000000000022), (14, 0.12000000000000022), (14, 0.12000000000000022), (14, 0.12000000000000022), (14, 0.12000000000000022), (14, 0.12000000000000022), (14, 0.12000000000000022), (14, 0.12000000000000022), (15, 0.12000000000000022), (15, 0.12000000000000022), (15, 0.12000000000000022), (16, 0.12000000000000022), (17, 0.12000000000000022), (17, 0.12000000000000022), (17, 0.12000000000000022), (17, 0.12000000000000022), (17, 0.12000000000000022), (17, 0.12000000000000022), (17, 0.12000000000000022), (18, 0.12000000000000022), (18, 0.12000000000000022), (18, 0.12000000000000022), (23, 0.12000000000000022), (23, 0.12000000000000022), (23, 0.12000000000000022), (28, 0.12000000000000022), (28, 0.12000000000000022), (28, 0.12000000000000022), (29, 0.12000000000000022), (29, 0.12000000000000022), (29, 0.12000000000000022), (33, 0.12000000000000022), (33, 0.12000000000000022), (36, 0.12000000000000022), (36, 0.12000000000000022), (36, 0.12000000000000022), (41, 0.12000000000000022), (41, 0.12000000000000022), (41, 0.12000000000000022), (11, 0.13), (11, 0.13), (11, 0.13), (11, 0.13), (11, 0.13), (12, 0.13), (12, 0.13), (12, 0.13), (12, 0.13), (18, 0.13), (18, 0.13), (18, 0.13), (18, 0.13), (18, 0.13), (19, 0.13), (19, 0.13), (24, 0.13), (24, 0.13), (28, 0.13), (28, 0.13), (28, 0.13), (28, 0.13), (30, 0.13), (29, 0.13), (29, 0.13), (33, 0.13), (33, 0.13), (33, 0.13), (36, 0.13), (36, 0.13), (36, 0.13), (36, 0.13), (41, 0.13), (41, 0.13), (10, 0.14), (10, 0.14), (10, 0.14), (10, 0.14), (10, 0.14), (19, 0.14), (19, 0.14), (19, 0.14), (19, 0.14), (19, 0.14), (20, 0.14), (20, 0.14), (20, 0.14), (24, 0.14), (24, 0.14), (28, 0.14), (28, 0.14), (29, 0.14), (30, 0.14), (30, 0.14), (30, 0.14), (33, 0.14), (34, 0.14), (36, 0.14), (36, 0.14), (41, 0.14), (41, 0.14), (9, 0.15000000000000002), (9, 0.15000000000000002), (9, 0.15000000000000002), (9, 0.15000000000000002), (9, 0.15000000000000002), (9, 0.15000000000000002), (9, 0.15000000000000002), (20, 0.15000000000000002), (20, 0.15000000000000002), (20, 0.15000000000000002), (20, 0.15000000000000002), (20, 0.15000000000000002), (21, 0.15000000000000002), (21, 0.15000000000000002), (24, 0.15000000000000002), (24, 0.15000000000000002), (29, 0.15000000000000002), (29, 0.15000000000000002), (30, 0.15000000000000002), (30, 0.15000000000000002), (30, 0.15000000000000002), (34, 0.15000000000000002), (34, 0.15000000000000002), (34, 0.15000000000000002), (34, 0.15000000000000002), (36, 0.15000000000000002), (36, 0.15000000000000002), (36, 0.15000000000000002), (36, 0.15000000000000002), (41, 0.15000000000000002), (41, 0.15000000000000002), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (8, 0.16000000000000003), (21, 0.16000000000000003), (21, 0.16000000000000003), (21, 0.16000000000000003), (21, 0.16000000000000003), (22, 0.16000000000000003), (25, 0.16000000000000003), (25, 0.16000000000000003), (29, 0.16000000000000003), (29, 0.16000000000000003), (29, 0.16000000000000003), (29, 0.16000000000000003), (30, 0.16000000000000003), (30, 0.16000000000000003), (30, 0.16000000000000003), (34, 0.16000000000000003), (34, 0.16000000000000003), (34, 0.16000000000000003), (36, 0.16000000000000003), (36, 0.16000000000000003), (36, 0.16000000000000003), (36, 0.16000000000000003), (41, 0.16000000000000003), (40, 0.16000000000000003), (7, 0.17000000000000015), (7, 0.17000000000000015), (7, 0.17000000000000015), (7, 0.17000000000000015), (7, 0.17000000000000015), (7, 0.17000000000000015), (8, 0.17000000000000015), (8, 0.17000000000000015), (22, 0.17000000000000015), (22, 0.17000000000000015), (22, 0.17000000000000015), (22, 0.17000000000000015), (25, 0.17000000000000015), (25, 0.17000000000000015), (25, 0.17000000000000015), (29, 0.17000000000000015), (29, 0.17000000000000015), (29, 0.17000000000000015), (30, 0.17000000000000015), (30, 0.17000000000000015), (30, 0.17000000000000015), (30, 0.17000000000000015), (34, 0.17000000000000015), (34, 0.17000000000000015), (34, 0.17000000000000015), (35, 0.17000000000000015), (35, 0.17000000000000015), (36, 0.17000000000000015), (36, 0.17000000000000015), (40, 0.17000000000000015), (40, 0.17000000000000015), (7, 0.18000000000000016), (7, 0.18000000000000016), (7, 0.18000000000000016), (7, 0.18000000000000016), (7, 0.18000000000000016), (7, 0.18000000000000016), (7, 0.18000000000000016), (22, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (25, 0.18000000000000016), (25, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (34, 0.18000000000000016), (34, 0.18000000000000016), (34, 0.18000000000000016), (34, 0.18000000000000016), (35, 0.18000000000000016), (35, 0.18000000000000016), (35, 0.18000000000000016), (35, 0.18000000000000016), (35, 0.18000000000000016), (40, 0.18000000000000016), (40, 0.18000000000000016), (40, 0.18000000000000016), (6, 0.18999999999999995), (6, 0.18999999999999995), (7, 0.18999999999999995), (7, 0.18999999999999995), (7, 0.18999999999999995), (23, 0.18999999999999995), (23, 0.18999999999999995), (23, 0.18999999999999995), (23, 0.18999999999999995), (24, 0.18999999999999995), (24, 0.18999999999999995), (24, 0.18999999999999995), (26, 0.18999999999999995), (26, 0.18999999999999995), (25, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (30, 0.18999999999999995), (34, 0.18999999999999995), (34, 0.18999999999999995), (34, 0.18999999999999995), (35, 0.18999999999999995), (35, 0.18999999999999995), (35, 0.18999999999999995), (35, 0.18999999999999995), (35, 0.18999999999999995), (35, 0.18999999999999995), (40, 0.18999999999999995), (40, 0.18999999999999995), (6, 0.20000000000000007), (6, 0.20000000000000007), (6, 0.20000000000000007), (6, 0.20000000000000007), (6, 0.20000000000000007), (24, 0.20000000000000007), (24, 0.20000000000000007), (24, 0.20000000000000007), (24, 0.20000000000000007), (24, 0.20000000000000007), (24, 0.20000000000000007), (24, 0.20000000000000007), (26, 0.20000000000000007), (26, 0.20000000000000007), (26, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (40, 0.20000000000000007), (40, 0.20000000000000007), (40, 0.20000000000000007), (6, 0.21000000000000008), (6, 0.21000000000000008), (6, 0.21000000000000008), (6, 0.21000000000000008), (6, 0.21000000000000008), (24, 0.21000000000000008), (25, 0.21000000000000008), (25, 0.21000000000000008), (25, 0.21000000000000008), (26, 0.21000000000000008), (26, 0.21000000000000008), (26, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (34, 0.21000000000000008), (40, 0.21000000000000008), (40, 0.21000000000000008), (40, 0.21000000000000008), (16, 0.22000000000000008), (16, 0.22000000000000008), (16, 0.22000000000000008), (15, 0.22000000000000008), (15, 0.22000000000000008), (15, 0.22000000000000008), (14, 0.22000000000000008), (14, 0.22000000000000008), (14, 0.22000000000000008), (13, 0.22000000000000008), (13, 0.22000000000000008), (13, 0.22000000000000008), (13, 0.22000000000000008), (13, 0.22000000000000008), (12, 0.22000000000000008), (12, 0.22000000000000008), (12, 0.22000000000000008), (12, 0.22000000000000008), (12, 0.22000000000000008), (12, 0.22000000000000008), (11, 0.22000000000000008), (11, 0.22000000000000008), (11, 0.22000000000000008), (11, 0.22000000000000008), (11, 0.22000000000000008), (11, 0.22000000000000008), (11, 0.22000000000000008), (6, 0.22000000000000008), (6, 0.22000000000000008), (6, 0.22000000000000008), (6, 0.22000000000000008), (25, 0.22000000000000008), (25, 0.22000000000000008), (25, 0.22000000000000008), (25, 0.22000000000000008), (26, 0.22000000000000008), (26, 0.22000000000000008), (26, 0.22000000000000008), (26, 0.22000000000000008), (40, 0.22000000000000008), (40, 0.22000000000000008), (40, 0.22000000000000008), (40, 0.22000000000000008), (18, 0.2300000000000002), (18, 0.2300000000000002), (17, 0.2300000000000002), (17, 0.2300000000000002), (17, 0.2300000000000002), (11, 0.2300000000000002), (11, 0.2300000000000002), (11, 0.2300000000000002), (11, 0.2300000000000002), (6, 0.2300000000000002), (6, 0.2300000000000002), (6, 0.2300000000000002), (6, 0.2300000000000002), (25, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (26, 0.2300000000000002), (40, 0.2300000000000002), (39, 0.2300000000000002), (39, 0.2300000000000002), (39, 0.2300000000000002), (19, 0.2400000000000002), (18, 0.2400000000000002), (11, 0.2400000000000002), (11, 0.2400000000000002), (11, 0.2400000000000002), (6, 0.2400000000000002), (6, 0.2400000000000002), (6, 0.2400000000000002), (26, 0.2400000000000002), (39, 0.2400000000000002), (39, 0.2400000000000002), (39, 0.2400000000000002), (19, 0.2500000000000001), (19, 0.2500000000000001), (11, 0.2500000000000001), (11, 0.2500000000000001), (6, 0.2500000000000001), (6, 0.2500000000000001), (6, 0.2500000000000001), (6, 0.2500000000000001), (39, 0.2500000000000001), (39, 0.2500000000000001), (39, 0.2500000000000001), (19, 0.26), (10, 0.26), (10, 0.26), (6, 0.26), (6, 0.26), (6, 0.26), (38, 0.26), (38, 0.26), (38, 0.26), (38, 0.26), (20, 0.27), (20, 0.27), (10, 0.27), (6, 0.27), (6, 0.27), (38, 0.27), (38, 0.27), (38, 0.27), (20, 0.28), (10, 0.28), (10, 0.28), (6, 0.28), (6, 0.28), (6, 0.28), (6, 0.28), (37, 0.28), (37, 0.28), (37, 0.28), (20, 0.29000000000000004), (10, 0.29000000000000004), (10, 0.29000000000000004), (6, 0.29000000000000004), (6, 0.29000000000000004), (6, 0.29000000000000004), (6, 0.29000000000000004), (37, 0.29000000000000004), (37, 0.29000000000000004), (37, 0.29000000000000004), (20, 0.30000000000000004), (10, 0.30000000000000004), (10, 0.30000000000000004), (6, 0.30000000000000004), (6, 0.30000000000000004), (37, 0.30000000000000004), (37, 0.30000000000000004), (37, 0.30000000000000004), (20, 0.30999999999999994), (10, 0.30999999999999994), (10, 0.30999999999999994), (6, 0.30999999999999994), (6, 0.30999999999999994), (6, 0.30999999999999994), (37, 0.30999999999999994), (37, 0.30999999999999994), (21, 0.32000000000000006), (21, 0.32000000000000006), (10, 0.32000000000000006), (10, 0.32000000000000006), (10, 0.32000000000000006), (7, 0.32000000000000006), (6, 0.32000000000000006), (6, 0.32000000000000006), (6, 0.32000000000000006), (6, 0.32000000000000006), (37, 0.32000000000000006), (21, 0.33000000000000007), (10, 0.33000000000000007), (10, 0.33000000000000007), (7, 0.33000000000000007), (7, 0.33000000000000007), (7, 0.33000000000000007), (7, 0.33000000000000007), (7, 0.33000000000000007), (7, 0.33000000000000007), (7, 0.33000000000000007), (37, 0.33000000000000007), (37, 0.33000000000000007), (21, 0.3400000000000001), (21, 0.3400000000000001), (10, 0.3400000000000001), (10, 0.3400000000000001), (10, 0.3400000000000001), (10, 0.3400000000000001), (10, 0.3400000000000001), (10, 0.3400000000000001), (10, 0.3400000000000001), (8, 0.3400000000000001), (7, 0.3400000000000001), (7, 0.3400000000000001), (7, 0.3400000000000001), (7, 0.3400000000000001), (7, 0.3400000000000001), (7, 0.3400000000000001), (37, 0.3400000000000001), (37, 0.3400000000000001), (37, 0.3400000000000001), (21, 0.3500000000000001), (21, 0.3500000000000001), (10, 0.3500000000000001), (10, 0.3500000000000001), (10, 0.3500000000000001), (10, 0.3500000000000001), (10, 0.3500000000000001), (10, 0.3500000000000001), (10, 0.3500000000000001), (9, 0.3500000000000001), (9, 0.3500000000000001), (9, 0.3500000000000001), (9, 0.3500000000000001), (9, 0.3500000000000001), (9, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (8, 0.3500000000000001), (37, 0.3500000000000001), (38, 0.3500000000000001), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (10, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (11, 0.3600000000000002), (12, 0.3600000000000002), (12, 0.3600000000000002), (12, 0.3600000000000002), (12, 0.3600000000000002), (21, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (9, 0.3600000000000002), (38, 0.3600000000000002), (38, 0.3600000000000002), (38, 0.3600000000000002), (10, 0.3700000000000002), (10, 0.3700000000000002), (12, 0.3700000000000002), (12, 0.3700000000000002), (12, 0.3700000000000002), (12, 0.3700000000000002), (12, 0.3700000000000002), (20, 0.3700000000000002), (20, 0.3700000000000002), (38, 0.3700000000000002), (38, 0.3700000000000002), (10, 0.3800000000000001), (10, 0.3800000000000001), (10, 0.3800000000000001), (10, 0.3800000000000001), (13, 0.3800000000000001), (13, 0.3800000000000001), (13, 0.3800000000000001), (13, 0.3800000000000001), (13, 0.3800000000000001), (13, 0.3800000000000001), (20, 0.3800000000000001), (20, 0.3800000000000001), (20, 0.3800000000000001), (38, 0.3800000000000001), (38, 0.3800000000000001), (10, 0.39), (13, 0.39), (13, 0.39), (13, 0.39), (13, 0.39), (20, 0.39), (20, 0.39), (38, 0.39), (38, 0.39), (10, 0.40000000000000013), (10, 0.40000000000000013), (14, 0.40000000000000013), (14, 0.40000000000000013), (14, 0.40000000000000013), (14, 0.40000000000000013), (20, 0.40000000000000013), (20, 0.40000000000000013), (20, 0.40000000000000013), (20, 0.40000000000000013), (38, 0.40000000000000013), (39, 0.40000000000000013), (10, 0.41000000000000003), (10, 0.41000000000000003), (14, 0.41000000000000003), (14, 0.41000000000000003), (14, 0.41000000000000003), (14, 0.41000000000000003), (15, 0.41000000000000003), (15, 0.41000000000000003), (19, 0.41000000000000003), (20, 0.41000000000000003), (20, 0.41000000000000003), (20, 0.41000000000000003), (39, 0.41000000000000003), (39, 0.41000000000000003), (10, 0.42000000000000004), (10, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (15, 0.42000000000000004), (16, 0.42000000000000004), (19, 0.42000000000000004), (19, 0.42000000000000004), (19, 0.42000000000000004), (19, 0.42000000000000004), (19, 0.42000000000000004), (19, 0.42000000000000004), (39, 0.42000000000000004), (39, 0.42000000000000004), (10, 0.43000000000000016), (10, 0.43000000000000016), (16, 0.43000000000000016), (16, 0.43000000000000016), (16, 0.43000000000000016), (16, 0.43000000000000016), (16, 0.43000000000000016), (16, 0.43000000000000016), (16, 0.43000000000000016), (17, 0.43000000000000016), (17, 0.43000000000000016), (17, 0.43000000000000016), (17, 0.43000000000000016), (17, 0.43000000000000016), (17, 0.43000000000000016), (17, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (18, 0.43000000000000016), (19, 0.43000000000000016), (19, 0.43000000000000016), (40, 0.43000000000000016), (40, 0.43000000000000016), (10, 0.44000000000000006), (10, 0.44000000000000006), (40, 0.44000000000000006), (10, 0.45000000000000007), (10, 0.45000000000000007), (40, 0.45000000000000007), (40, 0.45000000000000007), (10, 0.4600000000000001), (10, 0.4600000000000001), (10, 0.4600000000000001), (41, 0.4600000000000001), (10, 0.4700000000000001), (10, 0.4700000000000001), (41, 0.4700000000000001), (41, 0.4700000000000001), (11, 0.4800000000000002), (10, 0.4800000000000002), (10, 0.4800000000000002), (41, 0.4800000000000002), (11, 0.4900000000000001), (11, 0.4900000000000001), (42, 0.4900000000000001), (42, 0.4900000000000001), (11, 0.5), (11, 0.5), (26, 0.51), (26, 0.51), (26, 0.51), (26, 0.51), (26, 0.51), (26, 0.51), (26, 0.51), (26, 0.51), (25, 0.51), (25, 0.51), (25, 0.51), (25, 0.51), (25, 0.51), (25, 0.51), (25, 0.51), (25, 0.51), (11, 0.51), (11, 0.51), (11, 0.51), (43, 0.51), (27, 0.52), (27, 0.52), (27, 0.52), (27, 0.52), (26, 0.52), (25, 0.52), (25, 0.52), (25, 0.52), (25, 0.52), (24, 0.52), (24, 0.52), (12, 0.52), (12, 0.52), (12, 0.52), (43, 0.52), (28, 0.53), (28, 0.53), (27, 0.53), (27, 0.53), (24, 0.53), (24, 0.53), (24, 0.53), (24, 0.53), (24, 0.53), (24, 0.53), (12, 0.53), (12, 0.53), (43, 0.53), (28, 0.54), (28, 0.54), (28, 0.54), (24, 0.54), (23, 0.54), (23, 0.54), (23, 0.54), (23, 0.54), (13, 0.54), (13, 0.54), (44, 0.54), (29, 0.5500000000000002), (29, 0.5500000000000002), (23, 0.5500000000000002), (23, 0.5500000000000002), (23, 0.5500000000000002), (22, 0.5500000000000002), (13, 0.5500000000000002), (13, 0.5500000000000002), (13, 0.5500000000000002), (45, 0.5500000000000002), (29, 0.5599999999999999), (29, 0.5599999999999999), (22, 0.5599999999999999), (22, 0.5599999999999999), (22, 0.5599999999999999), (14, 0.5599999999999999), (14, 0.5599999999999999), (14, 0.5599999999999999), (45, 0.5599999999999999), (30, 0.57), (30, 0.57), (22, 0.57), (22, 0.57), (22, 0.57), (21, 0.57), (21, 0.57), (21, 0.57), (15, 0.57), (15, 0.57), (15, 0.57), (14, 0.57), (46, 0.57), (30, 0.5800000000000001), (21, 0.5800000000000001), (21, 0.5800000000000001), (21, 0.5800000000000001), (20, 0.5800000000000001), (20, 0.5800000000000001), (20, 0.5800000000000001), (20, 0.5800000000000001), (16, 0.5800000000000001), (16, 0.5800000000000001), (15, 0.5800000000000001), (15, 0.5800000000000001), (15, 0.5800000000000001), (31, 0.5900000000000001), (30, 0.5900000000000001), (20, 0.5900000000000001), (20, 0.5900000000000001), (19, 0.5900000000000001), (19, 0.5900000000000001), (19, 0.5900000000000001), (19, 0.5900000000000001), (19, 0.5900000000000001), (19, 0.5900000000000001), (18, 0.5900000000000001), (18, 0.5900000000000001), (18, 0.5900000000000001), (18, 0.5900000000000001), (18, 0.5900000000000001), (17, 0.5900000000000001), (17, 0.5900000000000001), (17, 0.5900000000000001), (17, 0.5900000000000001), (17, 0.5900000000000001), (16, 0.5900000000000001), (16, 0.5900000000000001), (16, 0.5900000000000001), (47, 0.5900000000000001), (31, 0.6000000000000002), (47, 0.6000000000000002), (31, 0.6099999999999999), (31, 0.62), (48, 0.62), (32, 0.63), (49, 0.63), (32, 0.6400000000000001), (32, 0.6500000000000001), (32, 0.6500000000000001), (33, 0.6600000000000001), (33, 0.6700000000000002), (33, 0.6800000000000002), (33, 0.69), (34, 0.7), (34, 0.7200000000000001), (34, 0.73), (34, 0.7400000000000001), (34, 0.75), (34, 0.77), (35, 0.7800000000000002), (35, 0.7900000000000003), (35, 0.8), (35, 0.8099999999999998), (35, 0.82), (35, 0.8300000000000001), (35, 0.84), (35, 0.8500000000000001), (35, 0.8600000000000002), (35, 0.8700000000000002), (35, 0.8799999999999999), (35, 0.8799999999999999), (35, 0.8900000000000001)]
sign2raw=  [(27, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (28, 0.0), (29, 0.0), (29, 0.0), (29, 0.0), (29, 0.0), (29, 0.0), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (19, 0.010000000000000009), (20, 0.010000000000000009), (20, 0.010000000000000009), (20, 0.010000000000000009), (20, 0.010000000000000009), (20, 0.010000000000000009), (21, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (27, 0.010000000000000009), (27, 0.010000000000000009), (29, 0.010000000000000009), (29, 0.010000000000000009), (30, 0.010000000000000009), (18, 0.020000000000000018), (18, 0.020000000000000018), (18, 0.020000000000000018), (18, 0.020000000000000018), (18, 0.020000000000000018), (18, 0.020000000000000018), (21, 0.020000000000000018), (21, 0.020000000000000018), (21, 0.020000000000000018), (26, 0.020000000000000018), (26, 0.020000000000000018), (26, 0.020000000000000018), (26, 0.020000000000000018), (30, 0.020000000000000018), (30, 0.020000000000000018), (30, 0.020000000000000018), (18, 0.030000000000000027), (18, 0.030000000000000027), (18, 0.030000000000000027), (18, 0.030000000000000027), (21, 0.030000000000000027), (22, 0.030000000000000027), (27, 0.030000000000000027), (27, 0.030000000000000027), (26, 0.030000000000000027), (30, 0.030000000000000027), (30, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (19, 0.040000000000000036), (19, 0.040000000000000036), (18, 0.040000000000000036), (18, 0.040000000000000036), (18, 0.040000000000000036), (18, 0.040000000000000036), (22, 0.040000000000000036), (22, 0.040000000000000036), (22, 0.040000000000000036), (22, 0.040000000000000036), (27, 0.040000000000000036), (27, 0.040000000000000036), (27, 0.040000000000000036), (30, 0.040000000000000036), (30, 0.040000000000000036), (31, 0.040000000000000036), (34, 0.040000000000000036), (35, 0.040000000000000036), (35, 0.040000000000000036), (35, 0.040000000000000036), (37, 0.040000000000000036), (37, 0.040000000000000036), (37, 0.040000000000000036), (37, 0.040000000000000036), (37, 0.040000000000000036), (19, 0.050000000000000044), (19, 0.050000000000000044), (19, 0.050000000000000044), (19, 0.050000000000000044), (19, 0.050000000000000044), (19, 0.050000000000000044), (19, 0.050000000000000044), (22, 0.050000000000000044), (23, 0.050000000000000044), (23, 0.050000000000000044), (23, 0.050000000000000044), (27, 0.050000000000000044), (27, 0.050000000000000044), (27, 0.050000000000000044), (31, 0.050000000000000044), (31, 0.050000000000000044), (31, 0.050000000000000044), (34, 0.050000000000000044), (34, 0.050000000000000044), (34, 0.050000000000000044), (37, 0.050000000000000044), (37, 0.050000000000000044), (37, 0.050000000000000044), (37, 0.050000000000000044), (20, 0.060000000000000164), (20, 0.060000000000000164), (20, 0.060000000000000164), (20, 0.060000000000000164), (20, 0.060000000000000164), (19, 0.060000000000000164), (19, 0.060000000000000164), (19, 0.060000000000000164), (23, 0.060000000000000164), (23, 0.060000000000000164), (23, 0.060000000000000164), (24, 0.060000000000000164), (24, 0.060000000000000164), (28, 0.060000000000000164), (27, 0.060000000000000164), (27, 0.060000000000000164), (31, 0.060000000000000164), (31, 0.060000000000000164), (31, 0.060000000000000164), (34, 0.060000000000000164), (34, 0.060000000000000164), (34, 0.060000000000000164), (37, 0.060000000000000164), (38, 0.060000000000000164), (38, 0.060000000000000164), (38, 0.060000000000000164), (21, 0.07000000000000017), (21, 0.07000000000000017), (20, 0.07000000000000017), (20, 0.07000000000000017), (20, 0.07000000000000017), (24, 0.07000000000000017), (24, 0.07000000000000017), (24, 0.07000000000000017), (24, 0.07000000000000017), (24, 0.07000000000000017), (28, 0.07000000000000017), (28, 0.07000000000000017), (28, 0.07000000000000017), (31, 0.07000000000000017), (31, 0.07000000000000017), (32, 0.07000000000000017), (34, 0.07000000000000017), (34, 0.07000000000000017), (34, 0.07000000000000017), (34, 0.07000000000000017), (38, 0.07000000000000017), (38, 0.07000000000000017), (38, 0.07000000000000017), (38, 0.07000000000000017), (38, 0.07000000000000017), (21, 0.07999999999999996), (21, 0.07999999999999996), (21, 0.07999999999999996), (21, 0.07999999999999996), (25, 0.07999999999999996), (25, 0.07999999999999996), (25, 0.07999999999999996), (25, 0.07999999999999996), (28, 0.07999999999999996), (28, 0.07999999999999996), (32, 0.07999999999999996), (32, 0.07999999999999996), (34, 0.07999999999999996), (34, 0.07999999999999996), (34, 0.07999999999999996), (38, 0.07999999999999996), (38, 0.07999999999999996), (38, 0.07999999999999996), (38, 0.07999999999999996), (38, 0.07999999999999996), (22, 0.08999999999999997), (22, 0.08999999999999997), (22, 0.08999999999999997), (25, 0.08999999999999997), (26, 0.08999999999999997), (26, 0.08999999999999997), (26, 0.08999999999999997), (29, 0.08999999999999997), (28, 0.08999999999999997), (32, 0.08999999999999997), (32, 0.08999999999999997), (32, 0.08999999999999997), (34, 0.08999999999999997), (34, 0.08999999999999997), (34, 0.08999999999999997), (38, 0.08999999999999997), (39, 0.08999999999999997), (39, 0.08999999999999997), (23, 0.09999999999999998), (23, 0.09999999999999998), (22, 0.09999999999999998), (22, 0.09999999999999998), (26, 0.09999999999999998), (26, 0.09999999999999998), (26, 0.09999999999999998), (26, 0.09999999999999998), (26, 0.09999999999999998), (27, 0.09999999999999998), (29, 0.09999999999999998), (29, 0.09999999999999998), (29, 0.09999999999999998), (32, 0.09999999999999998), (32, 0.09999999999999998), (34, 0.09999999999999998), (34, 0.09999999999999998), (34, 0.09999999999999998), (39, 0.09999999999999998), (39, 0.09999999999999998), (39, 0.09999999999999998), (23, 0.10999999999999999), (23, 0.10999999999999999), (23, 0.10999999999999999), (27, 0.10999999999999999), (27, 0.10999999999999999), (27, 0.10999999999999999), (27, 0.10999999999999999), (27, 0.10999999999999999), (27, 0.10999999999999999), (27, 0.10999999999999999), (29, 0.10999999999999999), (29, 0.10999999999999999), (29, 0.10999999999999999), (32, 0.10999999999999999), (33, 0.10999999999999999), (34, 0.10999999999999999), (34, 0.10999999999999999), (34, 0.10999999999999999), (39, 0.10999999999999999), (39, 0.10999999999999999), (39, 0.10999999999999999), (24, 0.1200000000000001), (24, 0.1200000000000001), (27, 0.1200000000000001), (27, 0.1200000000000001), (28, 0.1200000000000001), (28, 0.1200000000000001), (28, 0.1200000000000001), (30, 0.1200000000000001), (29, 0.1200000000000001), (29, 0.1200000000000001), (33, 0.1200000000000001), (33, 0.1200000000000001), (33, 0.1200000000000001), (34, 0.1200000000000001), (34, 0.1200000000000001), (39, 0.1200000000000001), (39, 0.1200000000000001), (24, 0.13000000000000012), (24, 0.13000000000000012), (24, 0.13000000000000012), (28, 0.13000000000000012), (28, 0.13000000000000012), (28, 0.13000000000000012), (28, 0.13000000000000012), (30, 0.13000000000000012), (30, 0.13000000000000012), (30, 0.13000000000000012), (33, 0.13000000000000012), (33, 0.13000000000000012), (33, 0.13000000000000012), (33, 0.13000000000000012), (34, 0.13000000000000012), (34, 0.13000000000000012), (34, 0.13000000000000012), (39, 0.13000000000000012), (39, 0.13000000000000012), (39, 0.13000000000000012), (25, 0.1399999999999999), (25, 0.1399999999999999), (25, 0.1399999999999999), (24, 0.1399999999999999), (28, 0.1399999999999999), (29, 0.1399999999999999), (29, 0.1399999999999999), (29, 0.1399999999999999), (30, 0.1399999999999999), (30, 0.1399999999999999), (33, 0.1399999999999999), (33, 0.1399999999999999), (35, 0.1399999999999999), (35, 0.1399999999999999), (35, 0.1399999999999999), (39, 0.1399999999999999), (39, 0.1399999999999999), (39, 0.1399999999999999), (39, 0.1399999999999999), (25, 0.15000000000000002), (25, 0.15000000000000002), (25, 0.15000000000000002), (29, 0.15000000000000002), (29, 0.15000000000000002), (29, 0.15000000000000002), (30, 0.15000000000000002), (30, 0.15000000000000002), (30, 0.15000000000000002), (33, 0.15000000000000002), (33, 0.15000000000000002), (34, 0.15000000000000002), (35, 0.15000000000000002), (35, 0.15000000000000002), (35, 0.15000000000000002), (39, 0.15000000000000002), (39, 0.15000000000000002), (39, 0.15000000000000002), (15, 0.16000000000000003), (15, 0.16000000000000003), (15, 0.16000000000000003), (15, 0.16000000000000003), (15, 0.16000000000000003), (15, 0.16000000000000003), (15, 0.16000000000000003), (16, 0.16000000000000003), (16, 0.16000000000000003), (16, 0.16000000000000003), (16, 0.16000000000000003), (16, 0.16000000000000003), (17, 0.16000000000000003), (17, 0.16000000000000003), (17, 0.16000000000000003), (17, 0.16000000000000003), (17, 0.16000000000000003), (18, 0.16000000000000003), (18, 0.16000000000000003), (18, 0.16000000000000003), (18, 0.16000000000000003), (18, 0.16000000000000003), (18, 0.16000000000000003), (19, 0.16000000000000003), (19, 0.16000000000000003), (19, 0.16000000000000003), (19, 0.16000000000000003), (19, 0.16000000000000003), (19, 0.16000000000000003), (20, 0.16000000000000003), (20, 0.16000000000000003), (20, 0.16000000000000003), (20, 0.16000000000000003), (20, 0.16000000000000003), (21, 0.16000000000000003), (21, 0.16000000000000003), (25, 0.16000000000000003), (25, 0.16000000000000003), (25, 0.16000000000000003), (29, 0.16000000000000003), (29, 0.16000000000000003), (29, 0.16000000000000003), (30, 0.16000000000000003), (30, 0.16000000000000003), (30, 0.16000000000000003), (30, 0.16000000000000003), (30, 0.16000000000000003), (34, 0.16000000000000003), (34, 0.16000000000000003), (34, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (39, 0.16000000000000003), (39, 0.16000000000000003), (39, 0.16000000000000003), (13, 0.17000000000000004), (13, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (14, 0.17000000000000004), (21, 0.17000000000000004), (21, 0.17000000000000004), (21, 0.17000000000000004), (21, 0.17000000000000004), (22, 0.17000000000000004), (22, 0.17000000000000004), (22, 0.17000000000000004), (22, 0.17000000000000004), (26, 0.17000000000000004), (26, 0.17000000000000004), (26, 0.17000000000000004), (30, 0.17000000000000004), (30, 0.17000000000000004), (30, 0.17000000000000004), (30, 0.17000000000000004), (30, 0.17000000000000004), (30, 0.17000000000000004), (30, 0.17000000000000004), (34, 0.17000000000000004), (34, 0.17000000000000004), (35, 0.17000000000000004), (35, 0.17000000000000004), (35, 0.17000000000000004), (39, 0.17000000000000004), (39, 0.17000000000000004), (39, 0.17000000000000004), (13, 0.18000000000000016), (13, 0.18000000000000016), (13, 0.18000000000000016), (13, 0.18000000000000016), (13, 0.18000000000000016), (13, 0.18000000000000016), (13, 0.18000000000000016), (13, 0.18000000000000016), (22, 0.18000000000000016), (22, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (23, 0.18000000000000016), (26, 0.18000000000000016), (26, 0.18000000000000016), (26, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (30, 0.18000000000000016), (34, 0.18000000000000016), (34, 0.18000000000000016), (34, 0.18000000000000016), (35, 0.18000000000000016), (35, 0.18000000000000016), (35, 0.18000000000000016), (39, 0.18000000000000016), (39, 0.18000000000000016), (12, 0.19000000000000017), (12, 0.19000000000000017), (12, 0.19000000000000017), (12, 0.19000000000000017), (12, 0.19000000000000017), (12, 0.19000000000000017), (13, 0.19000000000000017), (23, 0.19000000000000017), (24, 0.19000000000000017), (24, 0.19000000000000017), (24, 0.19000000000000017), (24, 0.19000000000000017), (24, 0.19000000000000017), (24, 0.19000000000000017), (24, 0.19000000000000017), (26, 0.19000000000000017), (26, 0.19000000000000017), (26, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (30, 0.19000000000000017), (34, 0.19000000000000017), (34, 0.19000000000000017), (34, 0.19000000000000017), (35, 0.19000000000000017), (35, 0.19000000000000017), (35, 0.19000000000000017), (39, 0.19000000000000017), (39, 0.19000000000000017), (39, 0.19000000000000017), (12, 0.20000000000000007), (12, 0.20000000000000007), (12, 0.20000000000000007), (12, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (25, 0.20000000000000007), (27, 0.20000000000000007), (27, 0.20000000000000007), (27, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (34, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (39, 0.20000000000000007), (39, 0.20000000000000007), (11, 0.20999999999999996), (11, 0.20999999999999996), (11, 0.20999999999999996), (11, 0.20999999999999996), (11, 0.20999999999999996), (25, 0.20999999999999996), (25, 0.20999999999999996), (25, 0.20999999999999996), (26, 0.20999999999999996), (26, 0.20999999999999996), (26, 0.20999999999999996), (26, 0.20999999999999996), (26, 0.20999999999999996), (26, 0.20999999999999996), (27, 0.20999999999999996), (27, 0.20999999999999996), (27, 0.20999999999999996), (27, 0.20999999999999996), (34, 0.20999999999999996), (34, 0.20999999999999996), (34, 0.20999999999999996), (35, 0.20999999999999996), (35, 0.20999999999999996), (35, 0.20999999999999996), (39, 0.20999999999999996), (39, 0.20999999999999996), (11, 0.21999999999999997), (11, 0.21999999999999997), (11, 0.21999999999999997), (11, 0.21999999999999997), (11, 0.21999999999999997), (26, 0.21999999999999997), (26, 0.21999999999999997), (26, 0.21999999999999997), (26, 0.21999999999999997), (26, 0.21999999999999997), (26, 0.21999999999999997), (26, 0.21999999999999997), (27, 0.21999999999999997), (27, 0.21999999999999997), (27, 0.21999999999999997), (27, 0.21999999999999997), (27, 0.21999999999999997), (34, 0.21999999999999997), (35, 0.21999999999999997), (35, 0.21999999999999997), (35, 0.21999999999999997), (35, 0.21999999999999997), (35, 0.21999999999999997), (35, 0.21999999999999997), (35, 0.21999999999999997), (39, 0.21999999999999997), (39, 0.21999999999999997), (39, 0.21999999999999997), (19, 0.22999999999999998), (18, 0.22999999999999998), (18, 0.22999999999999998), (18, 0.22999999999999998), (18, 0.22999999999999998), (18, 0.22999999999999998), (17, 0.22999999999999998), (17, 0.22999999999999998), (17, 0.22999999999999998), (17, 0.22999999999999998), (17, 0.22999999999999998), (16, 0.22999999999999998), (16, 0.22999999999999998), (16, 0.22999999999999998), (16, 0.22999999999999998), (16, 0.22999999999999998), (16, 0.22999999999999998), (16, 0.22999999999999998), (15, 0.22999999999999998), (15, 0.22999999999999998), (15, 0.22999999999999998), (11, 0.22999999999999998), (11, 0.22999999999999998), (11, 0.22999999999999998), (11, 0.22999999999999998), (11, 0.22999999999999998), (11, 0.22999999999999998), (11, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (27, 0.22999999999999998), (28, 0.22999999999999998), (28, 0.22999999999999998), (28, 0.22999999999999998), (28, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (35, 0.22999999999999998), (39, 0.22999999999999998), (39, 0.22999999999999998), (39, 0.22999999999999998), (21, 0.24), (20, 0.24), (20, 0.24), (20, 0.24), (20, 0.24), (20, 0.24), (19, 0.24), (19, 0.24), (15, 0.24), (15, 0.24), (15, 0.24), (15, 0.24), (15, 0.24), (15, 0.24), (11, 0.24), (11, 0.24), (11, 0.24), (11, 0.24), (11, 0.24), (11, 0.24), (27, 0.24), (27, 0.24), (27, 0.24), (27, 0.24), (27, 0.24), (28, 0.24), (28, 0.24), (28, 0.24), (28, 0.24), (28, 0.24), (35, 0.24), (35, 0.24), (39, 0.24), (39, 0.24), (22, 0.25), (21, 0.25), (21, 0.25), (15, 0.25), (15, 0.25), (14, 0.25), (11, 0.25), (11, 0.25), (11, 0.25), (11, 0.25), (11, 0.25), (11, 0.25), (39, 0.25), (39, 0.25), (39, 0.25), (22, 0.2599999999999999), (22, 0.2599999999999999), (22, 0.2599999999999999), (14, 0.2599999999999999), (10, 0.2599999999999999), (10, 0.2599999999999999), (10, 0.2599999999999999), (10, 0.2599999999999999), (10, 0.2599999999999999), (11, 0.2599999999999999), (11, 0.2599999999999999), (11, 0.2599999999999999), (39, 0.2599999999999999), (39, 0.2599999999999999), (23, 0.27), (22, 0.27), (14, 0.27), (14, 0.27), (10, 0.27), (10, 0.27), (10, 0.27), (39, 0.27), (39, 0.27), (39, 0.27), (39, 0.27), (23, 0.28), (23, 0.28), (14, 0.28), (10, 0.28), (10, 0.28), (10, 0.28), (10, 0.28), (39, 0.28), (39, 0.28), (39, 0.28), (38, 0.28), (23, 0.29000000000000004), (23, 0.29000000000000004), (14, 0.29000000000000004), (14, 0.29000000000000004), (14, 0.29000000000000004), (10, 0.29000000000000004), (10, 0.29000000000000004), (38, 0.29000000000000004), (38, 0.29000000000000004), (38, 0.29000000000000004), (38, 0.29000000000000004), (38, 0.29000000000000004), (24, 0.30000000000000004), (24, 0.30000000000000004), (14, 0.30000000000000004), (14, 0.30000000000000004), (10, 0.30000000000000004), (10, 0.30000000000000004), (10, 0.30000000000000004), (38, 0.30000000000000004), (38, 0.30000000000000004), (38, 0.30000000000000004), (38, 0.30000000000000004), (24, 0.31000000000000016), (24, 0.31000000000000016), (24, 0.31000000000000016), (14, 0.31000000000000016), (14, 0.31000000000000016), (14, 0.31000000000000016), (10, 0.31000000000000016), (10, 0.31000000000000016), (10, 0.31000000000000016), (10, 0.31000000000000016), (38, 0.31000000000000016), (38, 0.31000000000000016), (38, 0.31000000000000016), (38, 0.31000000000000016), (24, 0.3200000000000002), (24, 0.3200000000000002), (14, 0.3200000000000002), (14, 0.3200000000000002), (14, 0.3200000000000002), (11, 0.3200000000000002), (11, 0.3200000000000002), (11, 0.3200000000000002), (38, 0.3200000000000002), (38, 0.3200000000000002), (38, 0.3200000000000002), (25, 0.33000000000000007), (25, 0.33000000000000007), (24, 0.33000000000000007), (14, 0.33000000000000007), (14, 0.33000000000000007), (14, 0.33000000000000007), (11, 0.33000000000000007), (11, 0.33000000000000007), (11, 0.33000000000000007), (11, 0.33000000000000007), (38, 0.33000000000000007), (38, 0.33000000000000007), (38, 0.33000000000000007), (25, 0.33999999999999997), (25, 0.33999999999999997), (14, 0.33999999999999997), (14, 0.33999999999999997), (14, 0.33999999999999997), (11, 0.33999999999999997), (11, 0.33999999999999997), (11, 0.33999999999999997), (11, 0.33999999999999997), (38, 0.33999999999999997), (38, 0.33999999999999997), (38, 0.33999999999999997), (25, 0.3500000000000001), (25, 0.3500000000000001), (14, 0.3500000000000001), (14, 0.3500000000000001), (14, 0.3500000000000001), (14, 0.3500000000000001), (11, 0.3500000000000001), (11, 0.3500000000000001), (11, 0.3500000000000001), (11, 0.3500000000000001), (38, 0.3500000000000001), (38, 0.3500000000000001), (38, 0.3500000000000001), (38, 0.3500000000000001), (25, 0.36), (25, 0.36), (14, 0.36), (14, 0.36), (14, 0.36), (14, 0.36), (14, 0.36), (14, 0.36), (11, 0.36), (11, 0.36), (11, 0.36), (11, 0.36), (11, 0.36), (11, 0.36), (38, 0.36), (38, 0.36), (38, 0.36), (38, 0.36), (38, 0.36), (25, 0.37), (25, 0.37), (14, 0.37), (14, 0.37), (14, 0.37), (13, 0.37), (13, 0.37), (13, 0.37), (13, 0.37), (13, 0.37), (13, 0.37), (12, 0.37), (12, 0.37), (12, 0.37), (12, 0.37), (12, 0.37), (12, 0.37), (12, 0.37), (12, 0.37), (38, 0.37), (38, 0.37), (38, 0.37), (38, 0.37), (25, 0.3800000000000001), (25, 0.3800000000000001), (39, 0.3800000000000001), (39, 0.3800000000000001), (39, 0.3800000000000001), (39, 0.3800000000000001), (39, 0.3800000000000001), (39, 0.3800000000000001), (39, 0.3800000000000001), (24, 0.39), (24, 0.39), (24, 0.39), (39, 0.39), (39, 0.39), (39, 0.39), (39, 0.39), (39, 0.39), (24, 0.4), (24, 0.4), (39, 0.4), (39, 0.4), (40, 0.4), (24, 0.41000000000000003), (24, 0.41000000000000003), (40, 0.41000000000000003), (40, 0.41000000000000003), (40, 0.41000000000000003), (24, 0.42000000000000004), (24, 0.42000000000000004), (40, 0.42000000000000004), (13, 0.43000000000000016), (13, 0.43000000000000016), (13, 0.43000000000000016), (13, 0.43000000000000016), (13, 0.43000000000000016), (13, 0.43000000000000016), (13, 0.43000000000000016), (13, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (14, 0.43000000000000016), (15, 0.43000000000000016), (15, 0.43000000000000016), (15, 0.43000000000000016), (24, 0.43000000000000016), (24, 0.43000000000000016), (24, 0.43000000000000016), (24, 0.43000000000000016), (40, 0.43000000000000016), (41, 0.43000000000000016), (13, 0.44000000000000006), (13, 0.44000000000000006), (13, 0.44000000000000006), (15, 0.44000000000000006), (15, 0.44000000000000006), (15, 0.44000000000000006), (15, 0.44000000000000006), (15, 0.44000000000000006), (23, 0.44000000000000006), (23, 0.44000000000000006), (23, 0.44000000000000006), (24, 0.44000000000000006), (41, 0.44000000000000006), (41, 0.44000000000000006), (13, 0.44999999999999996), (13, 0.44999999999999996), (15, 0.44999999999999996), (16, 0.44999999999999996), (16, 0.44999999999999996), (16, 0.44999999999999996), (16, 0.44999999999999996), (16, 0.44999999999999996), (23, 0.44999999999999996), (23, 0.44999999999999996), (23, 0.44999999999999996), (23, 0.44999999999999996), (42, 0.44999999999999996), (42, 0.44999999999999996), (13, 0.45999999999999996), (13, 0.45999999999999996), (16, 0.45999999999999996), (16, 0.45999999999999996), (16, 0.45999999999999996), (16, 0.45999999999999996), (17, 0.45999999999999996), (17, 0.45999999999999996), (17, 0.45999999999999996), (22, 0.45999999999999996), (22, 0.45999999999999996), (22, 0.45999999999999996), (23, 0.45999999999999996), (23, 0.45999999999999996), (42, 0.45999999999999996), (13, 0.47), (13, 0.47), (17, 0.47), (17, 0.47), (17, 0.47), (18, 0.47), (21, 0.47), (22, 0.47), (22, 0.47), (22, 0.47), (22, 0.47), (42, 0.47), (43, 0.47), (13, 0.48), (13, 0.48), (18, 0.48), (18, 0.48), (18, 0.48), (18, 0.48), (21, 0.48), (21, 0.48), (21, 0.48), (21, 0.48), (21, 0.48), (21, 0.48), (43, 0.48), (14, 0.49), (18, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (19, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (20, 0.49), (21, 0.49), (43, 0.49), (44, 0.49), (14, 0.5000000000000001), (14, 0.5000000000000001), (44, 0.5000000000000001), (44, 0.5000000000000001), (14, 0.5099999999999999), (14, 0.5099999999999999), (45, 0.5099999999999999), (45, 0.5099999999999999), (14, 0.5199999999999999), (14, 0.5199999999999999), (45, 0.5199999999999999), (15, 0.53), (14, 0.53), (14, 0.53), (46, 0.53), (46, 0.53), (29, 0.54), (29, 0.54), (29, 0.54), (29, 0.54), (29, 0.54), (29, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (28, 0.54), (27, 0.54), (27, 0.54), (27, 0.54), (27, 0.54), (27, 0.54), (27, 0.54), (27, 0.54), (27, 0.54), (26, 0.54), (26, 0.54), (26, 0.54), (26, 0.54), (15, 0.54), (15, 0.54), (15, 0.54), (15, 0.54), (46, 0.54), (47, 0.54), (30, 0.5500000000000002), (29, 0.5500000000000002), (29, 0.5500000000000002), (29, 0.5500000000000002), (29, 0.5500000000000002), (26, 0.5500000000000002), (26, 0.5500000000000002), (25, 0.5500000000000002), (25, 0.5500000000000002), (25, 0.5500000000000002), (16, 0.5500000000000002), (16, 0.5500000000000002), (15, 0.5500000000000002), (15, 0.5500000000000002), (47, 0.5500000000000002), (30, 0.5599999999999998), (30, 0.5599999999999998), (30, 0.5599999999999998), (25, 0.5599999999999998), (25, 0.5599999999999998), (24, 0.5599999999999998), (24, 0.5599999999999998), (16, 0.5599999999999998), (16, 0.5599999999999998), (16, 0.5599999999999998), (16, 0.5599999999999998), (47, 0.5599999999999998), (30, 0.57), (30, 0.57), (30, 0.57), (24, 0.57), (24, 0.57), (24, 0.57), (23, 0.57), (23, 0.57), (23, 0.57), (17, 0.57), (17, 0.57), (17, 0.57), (48, 0.57), (48, 0.57), (31, 0.58), (30, 0.58), (30, 0.58), (23, 0.58), (22, 0.58), (22, 0.58), (22, 0.58), (22, 0.58), (22, 0.58), (21, 0.58), (21, 0.58), (21, 0.58), (21, 0.58), (20, 0.58), (20, 0.58), (20, 0.58), (20, 0.58), (19, 0.58), (19, 0.58), (19, 0.58), (19, 0.58), (19, 0.58), (18, 0.58), (18, 0.58), (18, 0.58), (18, 0.58), (17, 0.58), (48, 0.58), (31, 0.5900000000000001), (31, 0.5900000000000001), (49, 0.5900000000000001), (49, 0.5900000000000001), (31, 0.6000000000000001), (31, 0.6000000000000001), (31, 0.6000000000000001), (32, 0.6100000000000001), (32, 0.6100000000000001), (31, 0.6100000000000001), (32, 0.6200000000000001), (32, 0.6200000000000001), (32, 0.6300000000000001), (32, 0.6300000000000001), (32, 0.6300000000000001), (33, 0.6399999999999999), (33, 0.6399999999999999), (32, 0.6399999999999999), (33, 0.6499999999999999), (33, 0.6499999999999999), (33, 0.6499999999999999), (33, 0.6499999999999999), (33, 0.6600000000000001), (33, 0.6600000000000001), (33, 0.6600000000000001), (34, 0.67), (33, 0.67), (34, 0.6799999999999999), (34, 0.6799999999999999), (34, 0.6900000000000001), (34, 0.6900000000000001), (35, 0.7), (34, 0.7), (34, 0.7), (35, 0.7099999999999999), (35, 0.72), (35, 0.72), (35, 0.7300000000000002), (35, 0.7300000000000002), (36, 0.7400000000000002), (36, 0.7400000000000002), (36, 0.75), (36, 0.7599999999999998), (36, 0.7599999999999998), (36, 0.7699999999999999), (36, 0.78), (36, 0.7899999999999999), (37, 0.8), (37, 0.8), (37, 0.8), (37, 0.8100000000000002), (37, 0.8200000000000002), (37, 0.8200000000000002), (37, 0.8299999999999998), (37, 0.8299999999999998), (37, 0.8299999999999998)]
sign3raw= [(26, 0.0), (26, 0.0), (26, 0.0), (26, 0.0), (27, 0.0), (27, 0.0), (27, 0.0), (25, 0.010000000000000009), (25, 0.010000000000000009), (25, 0.010000000000000009), (25, 0.010000000000000009), (25, 0.010000000000000009), (25, 0.010000000000000009), (25, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (26, 0.010000000000000009), (27, 0.010000000000000009), (27, 0.010000000000000009), (27, 0.010000000000000009), (28, 0.010000000000000009), (28, 0.010000000000000009), (25, 0.020000000000000018), (25, 0.020000000000000018), (25, 0.020000000000000018), (25, 0.020000000000000018), (25, 0.020000000000000018), (28, 0.020000000000000018), (28, 0.020000000000000018), (28, 0.020000000000000018), (29, 0.020000000000000018), (29, 0.020000000000000018), (29, 0.020000000000000018), (26, 0.030000000000000027), (26, 0.030000000000000027), (26, 0.030000000000000027), (25, 0.030000000000000027), (25, 0.030000000000000027), (25, 0.030000000000000027), (29, 0.030000000000000027), (29, 0.030000000000000027), (29, 0.030000000000000027), (29, 0.030000000000000027), (30, 0.030000000000000027), (34, 0.030000000000000027), (34, 0.030000000000000027), (34, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (35, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (36, 0.030000000000000027), (26, 0.04000000000000015), (26, 0.04000000000000015), (26, 0.04000000000000015), (26, 0.04000000000000015), (26, 0.04000000000000015), (30, 0.04000000000000015), (30, 0.04000000000000015), (30, 0.04000000000000015), (30, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (34, 0.04000000000000015), (36, 0.04000000000000015), (36, 0.04000000000000015), (36, 0.04000000000000015), (36, 0.04000000000000015), (36, 0.04000000000000015), (36, 0.04000000000000015), (36, 0.04000000000000015), (27, 0.050000000000000155), (26, 0.050000000000000155), (26, 0.050000000000000155), (26, 0.050000000000000155), (26, 0.050000000000000155), (30, 0.050000000000000155), (31, 0.050000000000000155), (31, 0.050000000000000155), (34, 0.050000000000000155), (34, 0.050000000000000155), (34, 0.050000000000000155), (34, 0.050000000000000155), (34, 0.050000000000000155), (37, 0.050000000000000155), (37, 0.050000000000000155), (37, 0.050000000000000155), (37, 0.050000000000000155), (27, 0.05999999999999994), (27, 0.05999999999999994), (27, 0.05999999999999994), (27, 0.05999999999999994), (31, 0.05999999999999994), (31, 0.05999999999999994), (31, 0.05999999999999994), (34, 0.05999999999999994), (34, 0.05999999999999994), (34, 0.05999999999999994), (37, 0.05999999999999994), (37, 0.05999999999999994), (37, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (42, 0.05999999999999994), (43, 0.05999999999999994), (43, 0.05999999999999994), (43, 0.05999999999999994), (43, 0.05999999999999994), (43, 0.05999999999999994), (43, 0.05999999999999994), (43, 0.05999999999999994), (28, 0.07000000000000006), (28, 0.07000000000000006), (27, 0.07000000000000006), (27, 0.07000000000000006), (32, 0.07000000000000006), (32, 0.07000000000000006), (34, 0.07000000000000006), (34, 0.07000000000000006), (34, 0.07000000000000006), (37, 0.07000000000000006), (37, 0.07000000000000006), (38, 0.07000000000000006), (42, 0.07000000000000006), (42, 0.07000000000000006), (42, 0.07000000000000006), (42, 0.07000000000000006), (42, 0.07000000000000006), (42, 0.07000000000000006), (42, 0.07000000000000006), (43, 0.07000000000000006), (43, 0.07000000000000006), (43, 0.07000000000000006), (43, 0.07000000000000006), (44, 0.07000000000000006), (28, 0.08000000000000007), (28, 0.08000000000000007), (28, 0.08000000000000007), (28, 0.08000000000000007), (28, 0.08000000000000007), (32, 0.08000000000000007), (32, 0.08000000000000007), (32, 0.08000000000000007), (34, 0.08000000000000007), (34, 0.08000000000000007), (34, 0.08000000000000007), (38, 0.08000000000000007), (38, 0.08000000000000007), (38, 0.08000000000000007), (42, 0.08000000000000007), (42, 0.08000000000000007), (42, 0.08000000000000007), (42, 0.08000000000000007), (44, 0.08000000000000007), (44, 0.08000000000000007), (44, 0.08000000000000007), (29, 0.09000000000000008), (29, 0.09000000000000008), (28, 0.09000000000000008), (32, 0.09000000000000008), (32, 0.09000000000000008), (33, 0.09000000000000008), (34, 0.09000000000000008), (34, 0.09000000000000008), (34, 0.09000000000000008), (38, 0.09000000000000008), (38, 0.09000000000000008), (38, 0.09000000000000008), (42, 0.09000000000000008), (42, 0.09000000000000008), (42, 0.09000000000000008), (44, 0.09000000000000008), (44, 0.09000000000000008), (44, 0.09000000000000008), (29, 0.1000000000000002), (29, 0.1000000000000002), (29, 0.1000000000000002), (33, 0.1000000000000002), (33, 0.1000000000000002), (35, 0.1000000000000002), (34, 0.1000000000000002), (34, 0.1000000000000002), (38, 0.1000000000000002), (38, 0.1000000000000002), (38, 0.1000000000000002), (42, 0.1000000000000002), (42, 0.1000000000000002), (42, 0.1000000000000002), (44, 0.1000000000000002), (44, 0.1000000000000002), (45, 0.1000000000000002), (45, 0.1000000000000002), (30, 0.11000000000000021), (29, 0.11000000000000021), (33, 0.11000000000000021), (33, 0.11000000000000021), (33, 0.11000000000000021), (35, 0.11000000000000021), (35, 0.11000000000000021), (35, 0.11000000000000021), (35, 0.11000000000000021), (38, 0.11000000000000021), (38, 0.11000000000000021), (42, 0.11000000000000021), (42, 0.11000000000000021), (42, 0.11000000000000021), (45, 0.11000000000000021), (45, 0.11000000000000021), (30, 0.1200000000000001), (30, 0.1200000000000001), (30, 0.1200000000000001), (33, 0.1200000000000001), (33, 0.1200000000000001), (35, 0.1200000000000001), (35, 0.1200000000000001), (35, 0.1200000000000001), (38, 0.1200000000000001), (38, 0.1200000000000001), (38, 0.1200000000000001), (41, 0.1200000000000001), (42, 0.1200000000000001), (42, 0.1200000000000001), (45, 0.1200000000000001), (45, 0.1200000000000001), (45, 0.1200000000000001), (18, 0.13), (18, 0.13), (18, 0.13), (18, 0.13), (18, 0.13), (18, 0.13), (19, 0.13), (19, 0.13), (19, 0.13), (19, 0.13), (19, 0.13), (19, 0.13), (20, 0.13), (20, 0.13), (20, 0.13), (20, 0.13), (21, 0.13), (21, 0.13), (21, 0.13), (22, 0.13), (22, 0.13), (23, 0.13), (23, 0.13), (23, 0.13), (23, 0.13), (24, 0.13), (24, 0.13), (24, 0.13), (24, 0.13), (25, 0.13), (25, 0.13), (25, 0.13), (26, 0.13), (26, 0.13), (26, 0.13), (30, 0.13), (30, 0.13), (30, 0.13), (34, 0.13), (34, 0.13), (34, 0.13), (35, 0.13), (35, 0.13), (35, 0.13), (35, 0.13), (38, 0.13), (38, 0.13), (41, 0.13), (41, 0.13), (45, 0.13), (45, 0.13), (45, 0.13), (17, 0.14), (17, 0.14), (17, 0.14), (17, 0.14), (17, 0.14), (17, 0.14), (26, 0.14), (27, 0.14), (27, 0.14), (27, 0.14), (30, 0.14), (30, 0.14), (34, 0.14), (34, 0.14), (34, 0.14), (35, 0.14), (35, 0.14), (35, 0.14), (35, 0.14), (38, 0.14), (38, 0.14), (38, 0.14), (41, 0.14), (41, 0.14), (41, 0.14), (45, 0.14), (45, 0.14), (16, 0.15000000000000002), (16, 0.15000000000000002), (16, 0.15000000000000002), (16, 0.15000000000000002), (28, 0.15000000000000002), (28, 0.15000000000000002), (31, 0.15000000000000002), (31, 0.15000000000000002), (31, 0.15000000000000002), (34, 0.15000000000000002), (34, 0.15000000000000002), (35, 0.15000000000000002), (35, 0.15000000000000002), (35, 0.15000000000000002), (35, 0.15000000000000002), (38, 0.15000000000000002), (38, 0.15000000000000002), (41, 0.15000000000000002), (41, 0.15000000000000002), (41, 0.15000000000000002), (45, 0.15000000000000002), (45, 0.15000000000000002), (45, 0.15000000000000002), (16, 0.16000000000000003), (16, 0.16000000000000003), (16, 0.16000000000000003), (28, 0.16000000000000003), (29, 0.16000000000000003), (29, 0.16000000000000003), (31, 0.16000000000000003), (31, 0.16000000000000003), (31, 0.16000000000000003), (31, 0.16000000000000003), (34, 0.16000000000000003), (34, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (35, 0.16000000000000003), (38, 0.16000000000000003), (38, 0.16000000000000003), (38, 0.16000000000000003), (38, 0.16000000000000003), (41, 0.16000000000000003), (41, 0.16000000000000003), (41, 0.16000000000000003), (45, 0.16000000000000003), (45, 0.16000000000000003), (45, 0.16000000000000003), (16, 0.17000000000000004), (16, 0.17000000000000004), (29, 0.17000000000000004), (29, 0.17000000000000004), (31, 0.17000000000000004), (31, 0.17000000000000004), (31, 0.17000000000000004), (35, 0.17000000000000004), (35, 0.17000000000000004), (35, 0.17000000000000004), (35, 0.17000000000000004), (35, 0.17000000000000004), (38, 0.17000000000000004), (38, 0.17000000000000004), (38, 0.17000000000000004), (38, 0.17000000000000004), (38, 0.17000000000000004), (41, 0.17000000000000004), (41, 0.17000000000000004), (41, 0.17000000000000004), (41, 0.17000000000000004), (45, 0.17000000000000004), (45, 0.17000000000000004), (45, 0.17000000000000004), (15, 0.17999999999999994), (15, 0.17999999999999994), (30, 0.17999999999999994), (30, 0.17999999999999994), (31, 0.17999999999999994), (31, 0.17999999999999994), (31, 0.17999999999999994), (35, 0.17999999999999994), (35, 0.17999999999999994), (35, 0.17999999999999994), (35, 0.17999999999999994), (35, 0.17999999999999994), (35, 0.17999999999999994), (35, 0.17999999999999994), (38, 0.17999999999999994), (38, 0.17999999999999994), (38, 0.17999999999999994), (40, 0.17999999999999994), (41, 0.17999999999999994), (41, 0.17999999999999994), (45, 0.17999999999999994), (44, 0.17999999999999994), (44, 0.17999999999999994), (44, 0.17999999999999994), (23, 0.19000000000000006), (23, 0.19000000000000006), (22, 0.19000000000000006), (22, 0.19000000000000006), (22, 0.19000000000000006), (22, 0.19000000000000006), (22, 0.19000000000000006), (21, 0.19000000000000006), (21, 0.19000000000000006), (21, 0.19000000000000006), (21, 0.19000000000000006), (21, 0.19000000000000006), (20, 0.19000000000000006), (20, 0.19000000000000006), (20, 0.19000000000000006), (20, 0.19000000000000006), (20, 0.19000000000000006), (15, 0.19000000000000006), (15, 0.19000000000000006), (30, 0.19000000000000006), (30, 0.19000000000000006), (32, 0.19000000000000006), (32, 0.19000000000000006), (32, 0.19000000000000006), (32, 0.19000000000000006), (35, 0.19000000000000006), (35, 0.19000000000000006), (35, 0.19000000000000006), (35, 0.19000000000000006), (35, 0.19000000000000006), (35, 0.19000000000000006), (35, 0.19000000000000006), (38, 0.19000000000000006), (38, 0.19000000000000006), (38, 0.19000000000000006), (38, 0.19000000000000006), (38, 0.19000000000000006), (40, 0.19000000000000006), (40, 0.19000000000000006), (40, 0.19000000000000006), (40, 0.19000000000000006), (44, 0.19000000000000006), (44, 0.19000000000000006), (44, 0.19000000000000006), (44, 0.19000000000000006), (24, 0.20000000000000007), (24, 0.20000000000000007), (23, 0.20000000000000007), (23, 0.20000000000000007), (20, 0.20000000000000007), (20, 0.20000000000000007), (19, 0.20000000000000007), (19, 0.20000000000000007), (19, 0.20000000000000007), (15, 0.20000000000000007), (15, 0.20000000000000007), (15, 0.20000000000000007), (30, 0.20000000000000007), (31, 0.20000000000000007), (32, 0.20000000000000007), (32, 0.20000000000000007), (32, 0.20000000000000007), (32, 0.20000000000000007), (32, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (35, 0.20000000000000007), (38, 0.20000000000000007), (38, 0.20000000000000007), (38, 0.20000000000000007), (38, 0.20000000000000007), (40, 0.20000000000000007), (40, 0.20000000000000007), (40, 0.20000000000000007), (40, 0.20000000000000007), (40, 0.20000000000000007), (44, 0.20000000000000007), (44, 0.20000000000000007), (44, 0.20000000000000007), (44, 0.20000000000000007), (25, 0.21000000000000008), (24, 0.21000000000000008), (24, 0.21000000000000008), (24, 0.21000000000000008), (19, 0.21000000000000008), (19, 0.21000000000000008), (19, 0.21000000000000008), (19, 0.21000000000000008), (19, 0.21000000000000008), (15, 0.21000000000000008), (15, 0.21000000000000008), (15, 0.21000000000000008), (31, 0.21000000000000008), (31, 0.21000000000000008), (31, 0.21000000000000008), (31, 0.21000000000000008), (32, 0.21000000000000008), (32, 0.21000000000000008), (32, 0.21000000000000008), (32, 0.21000000000000008), (32, 0.21000000000000008), (38, 0.21000000000000008), (38, 0.21000000000000008), (38, 0.21000000000000008), (38, 0.21000000000000008), (38, 0.21000000000000008), (38, 0.21000000000000008), (39, 0.21000000000000008), (39, 0.21000000000000008), (39, 0.21000000000000008), (39, 0.21000000000000008), (39, 0.21000000000000008), (39, 0.21000000000000008), (40, 0.21000000000000008), (40, 0.21000000000000008), (43, 0.21000000000000008), (43, 0.21000000000000008), (43, 0.21000000000000008), (43, 0.21000000000000008), (43, 0.21000000000000008), (25, 0.22000000000000008), (25, 0.22000000000000008), (18, 0.22000000000000008), (18, 0.22000000000000008), (18, 0.22000000000000008), (14, 0.22000000000000008), (14, 0.22000000000000008), (14, 0.22000000000000008), (15, 0.22000000000000008), (31, 0.22000000000000008), (31, 0.22000000000000008), (31, 0.22000000000000008), (31, 0.22000000000000008), (32, 0.22000000000000008), (32, 0.22000000000000008), (32, 0.22000000000000008), (32, 0.22000000000000008), (32, 0.22000000000000008), (32, 0.22000000000000008), (32, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (38, 0.22000000000000008), (39, 0.22000000000000008), (39, 0.22000000000000008), (39, 0.22000000000000008), (39, 0.22000000000000008), (43, 0.22000000000000008), (43, 0.22000000000000008), (42, 0.22000000000000008), (26, 0.2300000000000002), (26, 0.2300000000000002), (25, 0.2300000000000002), (18, 0.2300000000000002), (18, 0.2300000000000002), (14, 0.2300000000000002), (14, 0.2300000000000002), (14, 0.2300000000000002), (14, 0.2300000000000002), (42, 0.2300000000000002), (42, 0.2300000000000002), (42, 0.2300000000000002), (42, 0.2300000000000002), (26, 0.2400000000000002), (26, 0.2400000000000002), (18, 0.2400000000000002), (18, 0.2400000000000002), (18, 0.2400000000000002), (15, 0.2400000000000002), (15, 0.2400000000000002), (15, 0.2400000000000002), (14, 0.2400000000000002), (42, 0.2400000000000002), (41, 0.2400000000000002), (41, 0.2400000000000002), (41, 0.2400000000000002), (27, 0.2500000000000001), (27, 0.2500000000000001), (18, 0.2500000000000001), (18, 0.2500000000000001), (18, 0.2500000000000001), (18, 0.2500000000000001), (18, 0.2500000000000001), (15, 0.2500000000000001), (15, 0.2500000000000001), (15, 0.2500000000000001), (15, 0.2500000000000001), (15, 0.2500000000000001), (41, 0.2500000000000001), (41, 0.2500000000000001), (41, 0.2500000000000001), (41, 0.2500000000000001), (27, 0.26), (27, 0.26), (18, 0.26), (18, 0.26), (18, 0.26), (18, 0.26), (18, 0.26), (17, 0.26), (17, 0.26), (17, 0.26), (15, 0.26), (15, 0.26), (15, 0.26), (15, 0.26), (15, 0.26), (15, 0.26), (40, 0.26), (40, 0.26), (40, 0.26), (27, 0.27000000000000013), (27, 0.27000000000000013), (27, 0.27000000000000013), (17, 0.27000000000000013), (17, 0.27000000000000013), (17, 0.27000000000000013), (17, 0.27000000000000013), (17, 0.27000000000000013), (17, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (16, 0.27000000000000013), (15, 0.27000000000000013), (15, 0.27000000000000013), (15, 0.27000000000000013), (40, 0.27000000000000013), (40, 0.27000000000000013), (40, 0.27000000000000013), (27, 0.28), (27, 0.28), (40, 0.28), (40, 0.28), (27, 0.29000000000000004), (27, 0.29000000000000004), (27, 0.29000000000000004), (40, 0.29000000000000004), (40, 0.29000000000000004), (16, 0.30000000000000016), (17, 0.30000000000000016), (17, 0.30000000000000016), (17, 0.30000000000000016), (17, 0.30000000000000016), (17, 0.30000000000000016), (17, 0.30000000000000016), (17, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (18, 0.30000000000000016), (19, 0.30000000000000016), (27, 0.30000000000000016), (27, 0.30000000000000016), (40, 0.30000000000000016), (40, 0.30000000000000016), (40, 0.30000000000000016), (40, 0.30000000000000016), (40, 0.30000000000000016), (16, 0.31000000000000005), (16, 0.31000000000000005), (16, 0.31000000000000005), (16, 0.31000000000000005), (16, 0.31000000000000005), (19, 0.31000000000000005), (19, 0.31000000000000005), (19, 0.31000000000000005), (19, 0.31000000000000005), (19, 0.31000000000000005), (27, 0.31000000000000005), (27, 0.31000000000000005), (27, 0.31000000000000005), (40, 0.31000000000000005), (40, 0.31000000000000005), (40, 0.31000000000000005), (16, 0.32000000000000006), (16, 0.32000000000000006), (16, 0.32000000000000006), (16, 0.32000000000000006), (16, 0.32000000000000006), (19, 0.32000000000000006), (19, 0.32000000000000006), (19, 0.32000000000000006), (19, 0.32000000000000006), (27, 0.32000000000000006), (27, 0.32000000000000006), (27, 0.32000000000000006), (40, 0.32000000000000006), (40, 0.32000000000000006), (40, 0.32000000000000006), (16, 0.33000000000000007), (16, 0.33000000000000007), (16, 0.33000000000000007), (20, 0.33000000000000007), (20, 0.33000000000000007), (27, 0.33000000000000007), (27, 0.33000000000000007), (27, 0.33000000000000007), (40, 0.33000000000000007), (40, 0.33000000000000007), (40, 0.33000000000000007), (16, 0.3400000000000001), (16, 0.3400000000000001), (16, 0.3400000000000001), (20, 0.3400000000000001), (20, 0.3400000000000001), (20, 0.3400000000000001), (27, 0.3400000000000001), (27, 0.3400000000000001), (40, 0.3400000000000001), (40, 0.3400000000000001), (16, 0.3500000000000002), (16, 0.3500000000000002), (16, 0.3500000000000002), (20, 0.3500000000000002), (21, 0.3500000000000002), (21, 0.3500000000000002), (21, 0.3500000000000002), (27, 0.3500000000000002), (27, 0.3500000000000002), (27, 0.3500000000000002), (41, 0.3500000000000002), (41, 0.3500000000000002), (41, 0.3500000000000002), (16, 0.3600000000000001), (16, 0.3600000000000001), (21, 0.3600000000000001), (21, 0.3600000000000001), (26, 0.3600000000000001), (26, 0.3600000000000001), (26, 0.3600000000000001), (26, 0.3600000000000001), (41, 0.3600000000000001), (41, 0.3600000000000001), (16, 0.37), (16, 0.37), (21, 0.37), (22, 0.37), (22, 0.37), (22, 0.37), (22, 0.37), (25, 0.37), (26, 0.37), (26, 0.37), (26, 0.37), (41, 0.37), (41, 0.37), (16, 0.38), (16, 0.38), (22, 0.38), (22, 0.38), (23, 0.38), (23, 0.38), (23, 0.38), (23, 0.38), (23, 0.38), (23, 0.38), (24, 0.38), (24, 0.38), (24, 0.38), (24, 0.38), (24, 0.38), (24, 0.38), (25, 0.38), (25, 0.38), (25, 0.38), (25, 0.38), (25, 0.38), (25, 0.38), (41, 0.38), (42, 0.38), (17, 0.39), (16, 0.39), (42, 0.39), (42, 0.39), (17, 0.4), (17, 0.4), (42, 0.4), (42, 0.4), (17, 0.41000000000000003), (17, 0.41000000000000003), (43, 0.41000000000000003), (43, 0.41000000000000003), (18, 0.42000000000000015), (17, 0.42000000000000015), (43, 0.42000000000000015), (18, 0.42999999999999994), (18, 0.42999999999999994), (43, 0.42999999999999994), (43, 0.42999999999999994), (44, 0.42999999999999994), (19, 0.43999999999999995), (18, 0.43999999999999995), (18, 0.43999999999999995), (44, 0.43999999999999995), (44, 0.43999999999999995), (31, 0.45000000000000007), (31, 0.45000000000000007), (31, 0.45000000000000007), (31, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (30, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (29, 0.45000000000000007), (28, 0.45000000000000007), (28, 0.45000000000000007), (28, 0.45000000000000007), (28, 0.45000000000000007), (28, 0.45000000000000007), (19, 0.45000000000000007), (19, 0.45000000000000007), (44, 0.45000000000000007), (45, 0.45000000000000007), (32, 0.4600000000000001), (32, 0.4600000000000001), (32, 0.4600000000000001), (32, 0.4600000000000001), (31, 0.4600000000000001), (31, 0.4600000000000001), (31, 0.4600000000000001), (31, 0.4600000000000001), (28, 0.4600000000000001), (28, 0.4600000000000001), (27, 0.4600000000000001), (27, 0.4600000000000001), (27, 0.4600000000000001), (27, 0.4600000000000001), (27, 0.4600000000000001), (19, 0.4600000000000001), (19, 0.4600000000000001), (19, 0.4600000000000001), (45, 0.4600000000000001), (45, 0.4600000000000001), (33, 0.4700000000000002), (33, 0.4700000000000002), (33, 0.4700000000000002), (33, 0.4700000000000002), (33, 0.4700000000000002), (32, 0.4700000000000002), (32, 0.4700000000000002), (32, 0.4700000000000002), (27, 0.4700000000000002), (27, 0.4700000000000002), (26, 0.4700000000000002), (26, 0.4700000000000002), (26, 0.4700000000000002), (26, 0.4700000000000002), (20, 0.4700000000000002), (20, 0.4700000000000002), (20, 0.4700000000000002), (45, 0.4700000000000002), (45, 0.4700000000000002), (34, 0.4799999999999999), (34, 0.4799999999999999), (34, 0.4799999999999999), (33, 0.4799999999999999), (33, 0.4799999999999999), (33, 0.4799999999999999), (33, 0.4799999999999999), (33, 0.4799999999999999), (33, 0.4799999999999999), (25, 0.4799999999999999), (25, 0.4799999999999999), (25, 0.4799999999999999), (25, 0.4799999999999999), (25, 0.4799999999999999), (24, 0.4799999999999999), (24, 0.4799999999999999), (21, 0.4799999999999999), (21, 0.4799999999999999), (20, 0.4799999999999999), (20, 0.4799999999999999), (46, 0.4799999999999999), (46, 0.4799999999999999), (34, 0.49), (34, 0.49), (34, 0.49), (34, 0.49), (34, 0.49), (24, 0.49), (24, 0.49), (24, 0.49), (23, 0.49), (23, 0.49), (23, 0.49), (23, 0.49), (22, 0.49), (22, 0.49), (22, 0.49), (22, 0.49), (22, 0.49), (21, 0.49), (21, 0.49), (46, 0.49), (34, 0.5), (34, 0.5), (34, 0.5), (46, 0.5), (47, 0.5), (35, 0.5100000000000001), (35, 0.5100000000000001), (34, 0.5100000000000001), (47, 0.5100000000000001), (47, 0.5100000000000001), (35, 0.5200000000000001), (35, 0.5200000000000001), (48, 0.5200000000000001), (35, 0.5300000000000002), (35, 0.5300000000000002), (35, 0.5300000000000002), (48, 0.5300000000000002), (48, 0.5300000000000002), (35, 0.54), (48, 0.54), (36, 0.5500000000000002), (36, 0.5500000000000002), (35, 0.5500000000000002), (49, 0.5500000000000002), (49, 0.5500000000000002), (36, 0.56), (36, 0.56), (49, 0.56), (36, 0.5699999999999998), (36, 0.5699999999999998), (36, 0.5800000000000002), (36, 0.5800000000000002), (36, 0.5800000000000002), (37, 0.5900000000000001), (37, 0.6), (37, 0.6100000000000001), (37, 0.6100000000000001), (37, 0.6199999999999999), (37, 0.6199999999999999), (37, 0.6299999999999999), (37, 0.64), (38, 0.6500000000000001), (38, 0.6500000000000001), (38, 0.6600000000000001), (38, 0.6699999999999999), (38, 0.6799999999999999), (38, 0.69), (38, 0.7000000000000001), (38, 0.71), (39, 0.7200000000000001), (39, 0.7200000000000001), (39, 0.7400000000000002), (39, 0.7400000000000002), (39, 0.7600000000000001), (39, 0.7600000000000001), (39, 0.7800000000000002), (39, 0.7800000000000002), (39, 0.7900000000000001), (39, 0.8000000000000002), (39, 0.8000000000000002), (39, 0.8099999999999998), (39, 0.8099999999999998)]

sign3= []
for i in range(len(sign3raw)):
    sign3.append([sign3raw[i][0], sign3raw[i][1]])
    


#_____________________________________________________________________________




#This is done because the functions were written in this way.  BEWARE!!!


#fountain (space printer) dimensions
fWidth *=kappa;
fHeight*=kappa;
#border areas
xfiller*=kappa 
yfiller*=kappa 
#Screen Dimensions
Width*=kappa
Height *=kappa

nLength *=kappa
cRadius*=kappa

nRadius=fWidth/2/(N+(N-1)*nSpacingOverNozz)/(1+nThickOverRad)
nThickness = nRadius*nThickOverRad;
nSpacing= 2*(nRadius+nThickness)*nSpacingOverNozz

dx = 2*(nThickness+nRadius)+nSpacing


In [5]:

Y1= FuncExpConverter(BottomCircle,[(fWidth/2.,fHeight/2.), cRadius, 0], dx, (fWidth/2-cRadius,fWidth/2+cRadius ))
Y2= FuncExpConverter(Line,[-1,fHeight,0], dx, (0,fWidth ))
Y3= FuncExpConverter(CenteredAbs,[1,0], dx, (0,fWidth ))
Y4= FuncExpConverter(CosA,[fHeight/5,5,0], dx, (0,fWidth))



# plt.plot(*zip(*Y1), 'r+')
# plt.show()
# print(Y1)

#______________________________SMILEY__________________________________________________________
# numValues=int(W.fWidth/W.dx) +1
x1 = np.arange(N)
x2= np.array(x1[1:-1])
#print (len(x1),len(x2))
y1 = TopCircle([(fWidth/2.,fHeight/2.),cRadius,x1*dx])
y2 = BottomCircle([(fWidth/2.,fHeight/2.),cRadius,x2*dx])
r = []
for i in range(len(x1)):
    r.append([x1[i],y1[i]])
for j in range(len(x2)):
    r.append([x2[j], y2[j]])
# import matplotlib.pyplot as plt  
# plt.plot(*zip(*r), 'r+')

smiley = copy.deepcopy(r)
smiley.append([int(N*(.5-.1)),.65*2*cRadius])
smiley.append([int(N*(.5+.1)),.65*2*cRadius])
x33 = np.array(x1[int(.375*N):int(.675*N)])
#print(x33)
y3 = BottomCircle([(fWidth/2.,fHeight/2.),cRadius/4.,x33*dx])
x3 = (x33-int(N/2))*2 +int(N/2)
#print (x3)
for aa in range(len(x3)):
    smiley.append([x3[aa], y3[aa]])



    
    

/Users/RxTenshou/anaconda3/envs/pygame1/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()


In [ ]:

    
if __name__ == '__main__':
    game = Waterfall(PhysVar,DimVar,Y3, PatternIsDelay=False)
    game.run()